In [ ]:
!pip install -q bitsandbytes peft accelerate


In [ ]:
!pip -q install huggingface_hub==0.25.2 faiss-cpu ir_datasets >/dev/null


In [ ]:
!pip install rank_bm25


In [ ]:
!pip -q install ir_datasets


In [ ]:
!pip install transformers -U

In [1]:
import os, json, math, random
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, TaskType

TRAIN_JSONL   = "/kaggle/input/final-dataset-train/instructions_one_best_per_id.jsonl"
OUT_DIR       = "/kaggle/working/mE5_lora8bit_infoNCE_v1"
BASE_MODEL    = "intfloat/multilingual-e5-base"    

SEED          = 42
EPOCHS        = 1
BATCH_SIZE    = 8        
ACCUM_STEPS   = 1
MAX_LEN       = 256
LR            = 1e-4      
WARMUP_RATIO  = 0.1
WEIGHT_DECAY  = 0.0
TEMP          = 0.05
MIX_HARD_NEGS = True
MAX_HNEG      = 4

LORA_R        = 16
LORA_ALPHA    = 32
LORA_DROPOUT  = 0.05
LORA_TARGETS  = ["query","key","value","dense"]   

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(SEED); torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

os.makedirs(OUT_DIR, exist_ok=True)
print("Device:", DEVICE)

def load_jsonl(path: str) -> List[Dict[str, Any]]:
    out = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line: continue
            try:
                out.append(json.loads(line))
            except:
                pass
    return out

rows = load_jsonl(TRAIN_JSONL)
print("Loaded rows:", len(rows))

class JSONLRetrievalDataset(Dataset):
    
    def __init__(self, data: List[Dict[str,Any]], max_hard_negs: int = MAX_HNEG):
        self.items = []
        for r in data:
            q = str(r.get("query_ru") or "").strip()
            inst = str(r.get("instruction") or "").strip()
            p = str(r.get("positive_ru") or "").strip()
            if not q or not p:
                continue
            negs = r.get("hard_negs_ru") or []
            negs = [str(x).strip() for x in negs if str(x).strip()][:max_hard_negs]
            self.items.append({
                "q": f"query: {q} {inst}".strip(),
                "p": f"passage: {p}".strip(),
                "negs": [f"passage: {t}" for t in negs]
            })
    def __len__(self): return len(self.items)
    def __getitem__(self, idx): return self.items[idx]

dataset = JSONLRetrievalDataset(rows)
print("Samples:", len(dataset))

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

@dataclass
class Batch:
    q: Dict[str, torch.Tensor]
    p: Dict[str, torch.Tensor]
    n: Optional[Dict[str, torch.Tensor]]
    B: int

def collate(items: List[Dict[str,Any]]) -> Batch:
    q_texts = [it["q"] for it in items]
    p_texts = [it["p"] for it in items]
    n_texts = [n for it in items for n in it["negs"]] if MIX_HARD_NEGS else []

    q = tok(q_texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    p = tok(p_texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    n = tok(n_texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt") if n_texts else None
    return Batch(q, p, n, len(items))

idx = list(range(len(dataset)))
random.Random(SEED).shuffle(idx)
valN = max(200, int(0.1 * len(idx)))
val_ids, train_ids = idx[:valN], idx[valN:]
train_loader = DataLoader(Subset(dataset, train_ids), batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=collate)
val_loader   = DataLoader(Subset(dataset, val_ids),   batch_size=32,       shuffle=False,                 collate_fn=collate)

def load_lora_8bit(model_name: str):
    torch.cuda.empty_cache()
    base = AutoModel.from_pretrained(
        model_name,
        load_in_8bit=True,
        device_map="auto",          
        low_cpu_mem_usage=True,
    )
    lcfg = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,   
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=LORA_TARGETS,             
        bias="none",
    )
    peft_model = get_peft_model(base, lcfg)
    peft_model.print_trainable_parameters()
    return peft_model

model = load_lora_8bit(BASE_MODEL)
model.train()

def mean_pooling(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    return (last_hidden_state * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)

def encode_batch(enc_dict: Dict[str, torch.Tensor]) -> torch.Tensor:
    dev = next(model.parameters()).device
    for k in enc_dict:
        enc_dict[k] = enc_dict[k].to(dev)
    out = model(**enc_dict).last_hidden_state
    emb = mean_pooling(out, enc_dict["attention_mask"])
    return F.normalize(emb, p=2, dim=1)

trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = bnb.optim.AdamW8bit(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY)

steps_per_epoch = max(1, len(train_loader) // ACCUM_STEPS)
total_steps = steps_per_epoch * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, int(WARMUP_RATIO * total_steps), total_steps)

ce = nn.CrossEntropyLoss()

def local_validate():
    model.eval()
    hit1=rec=mrr=ndcg=n=0.0
    with torch.no_grad():
        for b in val_loader:
            q = encode_batch(b.q)   # [B,d]
            p = encode_batch(b.p)   # [B,d]
            sims = (q @ p.t())      # [B,B]
            for i in range(sims.size(0)):
                row = sims[i]
                ranks = torch.argsort(row, descending=True)
                pos_rank = (ranks == i).nonzero(as_tuple=False).item() + 1
                k = 10
                hit1 += 1.0 if pos_rank == 1 else 0.0
                rec  += 1.0 if pos_rank <= k else 0.0
                mrr  += (1.0 / pos_rank) if pos_rank <= k else 0.0
                ndcg += (1.0 / math.log2(1 + pos_rank)) if pos_rank <= k else 0.0
                n += 1
    model.train()
    if n == 0: return {}
    return {"N": int(n), "Hit@1": hit1/n, "Recall@10": rec/n, "MRR@10": mrr/n, "nDCG@10": ndcg/n}

best_mrr = -1.0
global_step = 0

for epoch in range(1, EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS}")
    running = 0.0
    optimizer.zero_grad(set_to_none=True)
    pbar = tqdm(enumerate(train_loader, start=1), total=len(train_loader))

    for step_idx, b in pbar:
        q = encode_batch(b.q)                
        p = encode_batch(b.p)               
        logits = (q @ p.t()) / TEMP          
        labels = torch.arange(b.B, device=logits.device)
        loss = ce(logits, labels)

        if MIX_HARD_NEGS and b.n is not None and b.n["input_ids"].numel() > 0:
            n_emb = encode_batch(b.n)                       
            cand  = torch.cat([p, n_emb], dim=0)            
            logits2 = (q @ cand.t()) / TEMP
            loss2 = ce(logits2, labels)
            loss = 0.5 * loss + 0.5 * loss2

        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)

        if (step_idx % ACCUM_STEPS) == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

        running += loss.item()
        if step_idx % 50 == 0:
            pbar.set_postfix_str(f"loss={running/50:.4f} lr={scheduler.get_last_lr()[0]:.2e}")
            running = 0.0

    ep_dir = os.path.join(OUT_DIR, f"epoch{epoch}")
    os.makedirs(ep_dir, exist_ok=True)
    model.save_pretrained(ep_dir)
    tok.save_pretrained(ep_dir)

    metrics = local_validate()
    print("[VAL]", {k:(round(v,4) if isinstance(v,float) else v) for k,v in metrics.items()})
    if metrics and metrics.get("MRR@10", -1) > best_mrr:
        best_mrr = metrics["MRR@10"]
        best_dir = os.path.join(OUT_DIR, "best")
        os.makedirs(best_dir, exist_ok=True)
        model.save_pretrained(best_dir)
        tok.save_pretrained(best_dir)
        print(f"[BEST] MRR@10={best_mrr:.4f} saved -> {best_dir}")

model.save_pretrained(OUT_DIR)
tok.save_pretrained(OUT_DIR)
print("Saved to:", OUT_DIR)

2025-09-10 00:18:12.721125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757463492.951430     147 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757463493.015057     147 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device: cuda
Loaded rows: 2052
Samples: 2052


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

...

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

trainable params: 2,678,784 || all params: 280,722,432 || trainable%: 0.9542

Epoch 1/1


100%|██████████| 230/230 [03:06<00:00,  1.23it/s, loss=0.1943 lr=1.45e-05]


[VAL] {'N': 205, 'Hit@1': 0.9951, 'Recall@10': 1.0, 'MRR@10': 0.9976, 'nDCG@10': 0.9982}
[BEST] MRR@10=0.9976 saved -> /kaggle/working/mE5_lora8bit_infoNCE_v1/best
Saved to: /kaggle/working/mE5_lora8bit_infoNCE_v1


In [2]:
import os, json, math, random
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, TaskType

TRAIN_JSONL   = "/kaggle/input/final-dataset-train/instructions_one_best_per_id.jsonl"
OUT_DIR       = "/kaggle/working/mE5_lora8bit_queryonly_v1"
BASE_MODEL    = "intfloat/multilingual-e5-base"  

SEED          = 42
EPOCHS        = 1
BATCH_SIZE    = 8          
ACCUM_STEPS   = 1
MAX_LEN       = 256        
LR            = 1e-4       
WARMUP_RATIO  = 0.1
WEIGHT_DECAY  = 0.0
TEMP          = 0.05
MIX_HARD_NEGS = True
MAX_HNEG      = 4

LORA_R        = 16
LORA_ALPHA    = 32
LORA_DROPOUT  = 0.05
LORA_TARGETS  = ["query","key","value","dense"]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(SEED); torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

os.makedirs(OUT_DIR, exist_ok=True)
print("Device:", DEVICE)

def load_jsonl(path: str) -> List[Dict[str, Any]]:
    out = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line: continue
            try: out.append(json.loads(line))
            except: pass
    return out

rows = load_jsonl(TRAIN_JSONL)
print("Loaded rows:", len(rows))

class QueryOnlyRetrievalDataset(Dataset):

    def __init__(self, data: List[Dict[str,Any]], max_hard_negs: int = MAX_HNEG):
        self.items = []
        for r in data:
            q = str(r.get("query_ru") or "").strip()
            p = str(r.get("positive_ru") or "").strip()
            if not q or not p:
                continue
            negs = r.get("hard_negs_ru") or []
            negs = [str(x).strip() for x in negs if str(x).strip()][:max_hard_negs]
            self.items.append({
                "q": f"query: {q}".strip(),         
                "p": f"passage: {p}".strip(),
                "negs": [f"passage: {t}" for t in negs]
            })
    def __len__(self): return len(self.items)
    def __getitem__(self, idx): return self.items[idx]

dataset = QueryOnlyRetrievalDataset(rows)
print("Samples:", len(dataset))

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

@dataclass
class Batch:
    q: Dict[str, torch.Tensor]
    p: Dict[str, torch.Tensor]
    n: Optional[Dict[str, torch.Tensor]]
    B: int

def collate(items: List[Dict[str,Any]]) -> Batch:
    q_texts = [it["q"] for it in items]
    p_texts = [it["p"] for it in items]
    n_texts = [n for it in items for n in it["negs"]] if MIX_HARD_NEGS else []

    q = tok(q_texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    p = tok(p_texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    n = tok(n_texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt") if n_texts else None
    return Batch(q, p, n, len(items))

idx = list(range(len(dataset)))
random.Random(SEED).shuffle(idx)
valN = max(200, int(0.1 * len(idx)))
val_ids, train_ids = idx[:valN], idx[valN:]
train_loader = DataLoader(Subset(dataset, train_ids), batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=collate)
val_loader   = DataLoader(Subset(dataset, val_ids),   batch_size=32,       shuffle=False,                 collate_fn=collate)

def load_lora_8bit(model_name: str):
    torch.cuda.empty_cache()
    base = AutoModel.from_pretrained(
        model_name,
        load_in_8bit=True,
        device_map="auto",
        low_cpu_mem_usage=True,
    )
    lcfg = LoraConfig(
        task_type=TaskType.FEATURE_EXTRACTION,
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=LORA_TARGETS,
        bias="none",
    )
    peft_model = get_peft_model(base, lcfg)
    peft_model.print_trainable_parameters()
    return peft_model

model = load_lora_8bit(BASE_MODEL)
model.train()

def mean_pooling(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    return (last_hidden_state * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)

def encode_batch(enc_dict: Dict[str, torch.Tensor]) -> torch.Tensor:
    dev = next(model.parameters()).device
    for k in enc_dict:
        enc_dict[k] = enc_dict[k].to(dev)
    out = model(**enc_dict).last_hidden_state
    emb = mean_pooling(out, enc_dict["attention_mask"])
    return F.normalize(emb, p=2, dim=1)

trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = bnb.optim.AdamW8bit(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY)

steps_per_epoch = max(1, len(train_loader) // ACCUM_STEPS)
total_steps = steps_per_epoch * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, int(WARMUP_RATIO * total_steps), total_steps)

ce = nn.CrossEntropyLoss()

def local_validate():
    model.eval()
    hit1=rec=mrr=ndcg=n=0.0
    with torch.no_grad():
        for b in val_loader:
            q = encode_batch(b.q)   # [B,d]
            p = encode_batch(b.p)   # [B,d]
            sims = (q @ p.t())      # [B,B]
            for i in range(sims.size(0)):
                row = sims[i]
                ranks = torch.argsort(row, descending=True)
                pos_rank = (ranks == i).nonzero(as_tuple=False).item() + 1
                k = 10
                hit1 += 1.0 if pos_rank == 1 else 0.0
                rec  += 1.0 if pos_rank <= k else 0.0
                mrr  += (1.0 / pos_rank) if pos_rank <= k else 0.0
                ndcg += (1.0 / math.log2(1 + pos_rank)) if pos_rank <= k else 0.0
                n += 1
    model.train()
    if n == 0: return {}
    return {"N": int(n), "Hit@1": hit1/n, "Recall@10": rec/n, "MRR@10": mrr/n, "nDCG@10": ndcg/n}

best_mrr = -1.0
global_step = 0

for epoch in range(1, EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS}")
    running = 0.0
    optimizer.zero_grad(set_to_none=True)
    pbar = tqdm(enumerate(train_loader, start=1), total=len(train_loader))

    for step_idx, b in pbar:
        q = encode_batch(b.q)                
        p = encode_batch(b.p)                
        logits = (q @ p.t()) / TEMP          
        labels = torch.arange(b.B, device=logits.device)
        loss = ce(logits, labels)

        if MIX_HARD_NEGS and b.n is not None and b.n["input_ids"].numel() > 0:
            n_emb = encode_batch(b.n)                       
            cand  = torch.cat([p, n_emb], dim=0)            
            logits2 = (q @ cand.t()) / TEMP
            loss2 = ce(logits2, labels)
            loss = 0.5 * loss + 0.5 * loss2

        loss.backward()
        torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)

        if (step_idx % ACCUM_STEPS) == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

        running += loss.item()
        if step_idx % 50 == 0:
            pbar.set_postfix_str(f"loss={running/50:.4f} lr={scheduler.get_last_lr()[0]:.2e}")
            running = 0.0

    ep_dir = os.path.join(OUT_DIR, f"epoch{epoch}")
    os.makedirs(ep_dir, exist_ok=True)
    model.save_pretrained(ep_dir)
    tok.save_pretrained(ep_dir)

    metrics = local_validate()
    print("[VAL]", {k:(round(v,4) if isinstance(v,float) else v) for k,v in metrics.items()})
    if metrics and metrics.get("MRR@10", -1) > best_mrr:
        best_mrr = metrics["MRR@10"]
        best_dir = os.path.join(OUT_DIR, "best")
        os.makedirs(best_dir, exist_ok=True)
        model.save_pretrained(best_dir)
        tok.save_pretrained(best_dir)
        print(f"[BEST] MRR@10={best_mrr:.4f} saved -> {best_dir}")

model.save_pretrained(OUT_DIR)
tok.save_pretrained(OUT_DIR)
print("Saved to:", OUT_DIR)

Device: cuda
Loaded rows: 2052
Samples: 2052


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 2,678,784 || all params: 280,722,432 || trainable%: 0.9542

Epoch 1/1


100%|██████████| 230/230 [03:03<00:00,  1.25it/s, loss=0.4411 lr=1.45e-05]


[VAL] {'N': 205, 'Hit@1': 0.9854, 'Recall@10': 0.9951, 'MRR@10': 0.9902, 'nDCG@10': 0.9915}
[BEST] MRR@10=0.9902 saved -> /kaggle/working/mE5_lora8bit_queryonly_v1/best
Saved to: /kaggle/working/mE5_lora8bit_queryonly_v1


In [3]:
import ir_datasets as irds, json

out_dir = "mrtydi_ru_prepared"
import os; os.makedirs(out_dir, exist_ok=True)

ds = irds.load("mr-tydi/ru/test")  

with open(f"{out_dir}/corpus.jsonl", "w", encoding="utf-8") as fc:
    for d in ds.docs_iter():
        j = {"_id": str(d.doc_id), "title": getattr(d, "title", "") or "", "text": d.text or ""}
        fc.write(json.dumps(j, ensure_ascii=False) + "\n")

with open(f"{out_dir}/queries.jsonl", "w", encoding="utf-8") as fq:
    for q in ds.queries_iter():
        j = {"_id": str(q.query_id), "text": q.text}
        fq.write(json.dumps(j, ensure_ascii=False) + "\n")

with open(f"{out_dir}/qrels.tsv", "w", encoding="utf-8") as fr:
    for r in ds.qrels_iter():
        rel = int(getattr(r, "relevance", 1) > 0)
        fr.write(f"{r.query_id}\t{r.doc_id}\t{rel}\n")

print("Wrote:", out_dir, "->", os.listdir(out_dir))

[INFO] If you have a local copy of https://git.uwaterloo.ca/jimmylin/mr.tydi/-/raw/master/data/mrtydi-v1.0-russian.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/fab64459133bc93a0bec2f0559bfb423
[INFO] [starting] https://git.uwaterloo.ca/jimmylin/mr.tydi/-/raw/master/data/mrtydi-v1.0-russian.tar.gz
[INFO] [finished] https://git.uwaterloo.ca/jimmylin/mr.tydi/-/raw/master/data/mrtydi-v1.0-russian.tar.gz: [00:25] [1.55GB] [61.0MB/s]
[INFO] [starting] extracting from tar file                                                                         
[INFO] [finished] extracting from tar file [2.52s]


Wrote: mrtydi_ru_prepared -> ['corpus.jsonl', 'qrels.tsv', 'queries.jsonl']


In [7]:
import os, json, random

DATA_DIR = "/kaggle/working/mrtydi_ru_prepared"  # same as in your eval script
Q_PATH   = os.path.join(DATA_DIR, "queries.jsonl")
OUT_PATH = os.path.join(DATA_DIR, "instructions.jsonl")

TEMPLATES = [
    "Ты — поисковая система. Найди наиболее релевантные документы, которые помогают ответить на запрос. Возвращай документы, а не готовый ответ.",
    "Найди статьи/пассажи, содержащие информацию для ответа на вопрос. Отдавай приоритет точности и контекстной релевантности.",
    "Выполни поиск по корпусу и верни документы с наибольшей вероятностью содержать ответ. Избегай нерелевантных совпадений."
]

random.seed(42)
n = 0
with open(Q_PATH, "r", encoding="utf-8") as fin, open(OUT_PATH, "w", encoding="utf-8") as fout:
    for line in fin:
        j = json.loads(line)
        qid = str(j.get("_id") or j.get("id"))
        if not qid: 
            continue
        instr = TEMPLATES[hash(qid) % len(TEMPLATES)]   # stable pseudo-random choice per qid
        out = {"_id": qid, "instruction": instr}
        fout.write(json.dumps(out, ensure_ascii=False) + "\n")
        n += 1

print(f"Wrote {n} instructions to:", OUT_PATH)


Wrote 995 instructions to: /kaggle/working/mrtydi_ru_prepared/instructions.jsonl


In [8]:
# Mr.TyDi-RU eval (BM25 + Base + LoRA adapters)
#  - p-MRR@k = MRR@k(q+instr) − MRR@k(q) 
# ===========================================================
import os, io, json, gzip, math, random, re, time, hashlib
from collections import defaultdict
from typing import Dict, List, Tuple

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel

DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE_DOC = 64
BATCH_SIZE_Q   = 64
MAX_LEN        = 256
K_LIST         = [1, 5, 10]
SEED           = 42

DOC_CAP     = 20000     
FAST_Q_CAP  = None       

USE_PREFIXES = True

# Models
BASE_MODEL          = "intfloat/multilingual-e5-base"
FINETUNED_INST_DIR  = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"       
FINETUNED_QUERY_DIR = "/kaggle/working/mE5_lora8bit_queryonly_v1/best"    

LOCAL_DATA_DIR = "/kaggle/working/mrtydi_ru_prepared"
INSTR_JSONL    = os.path.join(LOCAL_DATA_DIR, "instructions.jsonl")

ROOT_OUT  = "/kaggle/working/mrtydi_ru_eval"
EMB_CACHE = os.path.join(ROOT_OUT, "emb_cache")
os.makedirs(ROOT_OUT, exist_ok=True)
os.makedirs(EMB_CACHE, exist_ok=True)

random.seed(SEED); np.random.seed(SEED)
print(f"Device: {DEVICE} | DOC_CAP={DOC_CAP} | FAST_Q_CAP={FAST_Q_CAP}")

def tic(msg: str):
    print(msg, end="", flush=True)
    return time.time()

def toc(t0: float, prefix: str = "done"):
    dt = time.time() - t0
    print(f" {prefix} in {dt:.1f}s.")

def smart_open(path):
    if path.endswith(".gz"):
        f = gzip.open(path, "rb")
        return io.TextIOWrapper(f, encoding="utf-8")
    return open(path, "r", encoding="utf-8")

def _tqdm_lines(path: str, desc: str):
    """Yield lines and update tqdm by bytes read to show ETA without pre-counting lines."""
    total = os.path.getsize(path) if os.path.exists(path) else None
    with smart_open(path) as f, tqdm(total=total, unit="B", unit_scale=True, desc=desc) as pbar:
        for line in f:
            pbar.update(len(line.encode("utf-8")))
            yield line

def load_corpus(path: str) -> Dict[str, str]:
    corpus = {}
    for line in _tqdm_lines(path, "Loading corpus"):
        j = json.loads(line)
        cid   = str(j.get("_id") or j.get("id"))
        title = (j.get("title") or "").strip()
        text  = (j.get("text")  or "").strip()
        corpus[cid] = (title + " " + text).strip()
    return corpus

def load_queries(path: str) -> Dict[str, str]:
    queries = {}
    for line in _tqdm_lines(path, "Loading queries"):
        j = json.loads(line)
        qid  = str(j.get("_id") or j.get("id"))
        text = (j.get("text") or "").strip()
        queries[qid] = text
    return queries

def load_instructions_optional(path: str) -> Dict[str, str]:
    """Optional JSONL with instruction text per qid. Tries several key names."""
    if not os.path.exists(path):
        return {}
    inst_map = {}
    for line in _tqdm_lines(path, "Loading instructions"):
        j = json.loads(line)
        qid = str(j.get("_id") or j.get("id") or j.get("qid") or "").strip()
        instr = (j.get("instruction") or j.get("instruction_og") or j.get("instr") or "").strip()
        if qid and instr:
            inst_map[qid] = instr
    return inst_map

def load_qrels_tsv(path: str) -> Dict[str, Dict[str, int]]:
    qrels = defaultdict(dict)
    for row in _tqdm_lines(path, "Loading qrels"):
        row = row.strip()
        if not row or row.startswith("#"): continue
        parts = row.split("\t")
        if len(parts) < 3: continue
        qid, did, rel = parts[0].strip(), parts[1].strip(), parts[2].strip()
        try:
            rel = int(float(rel))
        except:
            continue
        if rel > 0:
            qrels[qid][did] = 1
    return dict(qrels)

for fn in ["corpus.jsonl", "queries.jsonl", "qrels.tsv"]:
    if not os.path.exists(os.path.join(LOCAL_DATA_DIR, fn)):
        raise FileNotFoundError(f"Missing {fn} under {LOCAL_DATA_DIR}")

t0 = tic("Loading dataset files…")
corpus  = load_corpus(os.path.join(LOCAL_DATA_DIR, "corpus.jsonl"))
queries = load_queries(os.path.join(LOCAL_DATA_DIR, "queries.jsonl"))
qrels   = load_qrels_tsv(os.path.join(LOCAL_DATA_DIR, "qrels.tsv"))
instructions = load_instructions_optional(INSTR_JSONL)  # may be empty
toc(t0, prefix=f"Loaded (docs={len(corpus):,}, queries={len(queries):,}, qrels_q={len(qrels):,}, instr_q={len(instructions):,})")

rng = random.Random(SEED)

qids_all = [qid for qid in qrels.keys() if qid in queries]

all_pos_doc_ids = set()
for qid in qids_all:
    all_pos_doc_ids.update(qrels[qid].keys())

pos_doc_ids = [d for d in all_pos_doc_ids if d in corpus]

if DOC_CAP is None:
    doc_ids = list(corpus.keys())
    neg_added = "N/A"
else:
    if len(pos_doc_ids) >= DOC_CAP:
        rng.shuffle(pos_doc_ids)
        doc_ids = pos_doc_ids[:DOC_CAP]
        neg_added = 0
    else:
        need = DOC_CAP - len(pos_doc_ids)
        neg_pool = [d for d in corpus.keys() if d not in all_pos_doc_ids]
        rng.shuffle(neg_pool)
        neg_added = min(need, len(neg_pool))
        doc_ids = pos_doc_ids + neg_pool[:neg_added]

corpus = {d: corpus[d] for d in doc_ids}
doc_texts_raw  = [corpus[d] for d in doc_ids]

qids_all = [qid for qid in qids_all if any((did in corpus) for did in qrels[qid].keys())]

if FAST_Q_CAP and FAST_Q_CAP < len(qids_all):
    rng.shuffle(qids_all)
    qids_all = qids_all[:FAST_Q_CAP]

num_q_with_pos = len(qids_all)
print(
    f"Eval set -> docs: {len(doc_ids):,} "
    f"(positives kept: {len(pos_doc_ids):,}, random negatives added: {neg_added}) "
    f"| queries: {num_q_with_pos:,}"
)
if num_q_with_pos == 0:
    raise RuntimeError("No queries left after subsetting. Increase DOC_CAP, disable caps, or verify your qrels/doc ids.")

def add_prefix(s: str, kind: str) -> str:
    if not USE_PREFIXES: return s
    return (("query: " if kind == "query" else "passage: ") + s).strip()

doc_texts_pref = [add_prefix(t, "passage") for t in doc_texts_raw]
query_map_raw  = {qid: queries[qid] for qid in qids_all}
query_map_pref_plain = {qid: add_prefix(queries[qid], "query") for qid in qids_all}

qids_with_instr = [qid for qid in qids_all if qid in instructions]
query_map_pref_instr = {qid: add_prefix((queries[qid] + " " + instructions[qid]).strip(), "query")
                        for qid in qids_with_instr}

def eval_multi_k(run, qrels_map, ks):
    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks:
        stats.update({f"Recall@{k}":0.0, f"MRR@{k}":0.0, f"nDCG@{k}":0.0})
    n = 0
    for qid, ranking in run.items():
        if qid not in qrels_map: continue
        n += 1
        rels = qrels_map[qid]
        pos_ranks = [r for r,(did,_) in enumerate(ranking, start=1) if did in rels]
        pos_ranks.sort()
        if pos_ranks and pos_ranks[0] == 1:
            stats["Hit@1"] += 1.0
        for k in ks:
            stats[f"Recall@{k}"] += float(any(r <= k for r in pos_ranks))
            stats[f"MRR@{k}"]    += (1.0/pos_ranks[0]) if (pos_ranks and pos_ranks[0] <= k) else 0.0
            dcg  = sum(1.0/math.log2(r+1) for r in pos_ranks if r <= k)
            m    = len(rels)
            idcg = sum(1.0/math.log2(r+1) for r in range(1, min(m, k)+1))
            stats[f"nDCG@{k}"] += (dcg/idcg) if idcg > 0 else 0.0
    if n == 0:
        return stats
    stats["N"] = n
    for k in ["Hit@1"]+[f"Recall@{x}" for x in ks]+[f"MRR@{x}" for x in ks]+[f"nDCG@{x}" for x in ks]:
        stats[k] /= n
    return stats

def fmt(d): 
    return {k:(round(v,4) if isinstance(v,float) else v) for k,v in d.items()}

def pmrr(metrics_plain: Dict[str,float], metrics_instr: Dict[str,float], ks: List[int]) -> Dict[str,float]:
    out = {}
    for k in ks:
        a = metrics_plain.get(f"MRR@{k}", 0.0)
        b = metrics_instr.get(f"MRR@{k}", 0.0)
        out[f"pMRR@{k}"] = b - a
    return out

def mean_pool(last_hidden, attention_mask):
    m = attention_mask.unsqueeze(-1).expand_as(last_hidden).float()
    return (last_hidden * m).sum(1) / m.sum(1).clamp(min=1e-9)

def corpus_signature(doc_ids: List[str]) -> str:
    head = "||".join(doc_ids[:50]); tail = "||".join(doc_ids[-50:]) if len(doc_ids) > 50 else ""
    return hashlib.md5((head + "|" + tail + f"|{len(doc_ids)}").encode()).hexdigest()[:8]

def signature(model_or_adapter: str) -> str:
    name = os.path.basename(model_or_adapter.rstrip("/")) if os.path.isdir(model_or_adapter) else model_or_adapter.replace("/", "_")
    return f"{corpus_signature(doc_ids)}__L{MAX_LEN}__{name}"

def load_tok_model_any(model_or_adapter: str, base_model: str):
    """Load tokenizer + model"""
    is_adapter = os.path.isdir(model_or_adapter) and os.path.exists(os.path.join(model_or_adapter, "adapter_config.json"))
    tok_src = model_or_adapter if (os.path.isdir(model_or_adapter) and os.path.exists(os.path.join(model_or_adapter, "tokenizer.json"))) else base_model
    tok = AutoTokenizer.from_pretrained(tok_src, use_fast=True)

    def _load_base(to_device: str):
        dtype = torch.float16 if to_device == "cuda" else None
        return AutoModel.from_pretrained(base_model, torch_dtype=dtype, low_cpu_mem_usage=True).to(to_device).eval()

    try:
        base = _load_base("cuda" if DEVICE == "cuda" else "cpu")
    except RuntimeError as e:
        print(f"[warn] GPU load failed ({type(e).__name__}): falling back to CPU.")
        base = _load_base("cpu")

    if is_adapter:
        mdl = PeftModel.from_pretrained(base, model_or_adapter).eval()
    else:
        mdl = base
    return tok, mdl

@torch.no_grad()
def encode_texts(mdl, tok, texts: List[str], bs: int, max_len: int) -> torch.Tensor:
    dev = next(mdl.parameters()).device
    outs = []
    for i in tqdm(range(0, len(texts), bs), total=(len(texts)+bs-1)//bs, desc=f"Encode@{dev}"):
        batch = texts[i:i+bs]
        enc = tok(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        for k in enc: enc[k] = enc[k].to(dev)
        last = mdl(**enc).last_hidden_state
        pooled = mean_pool(last, enc["attention_mask"])
        outs.append(F.normalize(pooled, p=2, dim=1).cpu())
    return torch.cat(outs, 0) if outs else torch.empty(0, mdl.config.hidden_size)

def build_index(doc_embs_cpu: torch.Tensor):
    try:
        import faiss
        index = faiss.IndexFlatIP(doc_embs_cpu.shape[1])
        index.add(doc_embs_cpu.numpy().astype("float32"))
        return index, True
    except Exception:
        return doc_embs_cpu, False

@torch.no_grad()
def search_index(index, is_faiss: bool, q_embs: torch.Tensor, topk: int):
    if is_faiss:
        import faiss
        D, I = index.search(q_embs.numpy().astype("float32"), topk)
        return I, D
    sims = torch.mm(q_embs, index.t())
    D, I = torch.topk(sims, k=min(topk, sims.size(1)), dim=1)
    return I.numpy(), D.numpy()

def eval_model(model_or_adapter: str, label: str,
               queries_map_pref: Dict[str,str], qids_list: List[str]):
    print(f"\n=== {label} ===")
    t0 = tic("Loading model/tokenizer…")
    tok, mdl = load_tok_model_any(model_or_adapter, BASE_MODEL)
    toc(t0)

    sig = signature(model_or_adapter)
    cache_path = os.path.join(EMB_CACHE, f"docs__{sig}.pt")

    if os.path.exists(cache_path):
        doc_embs = torch.load(cache_path, map_location="cpu")
        print(f"[cache] doc embs loaded: {doc_embs.shape}")
    else:
        t1 = tic(f"Encoding docs (N={len(doc_texts_pref):,})…")
        doc_embs = encode_texts(mdl, tok, doc_texts_pref, BATCH_SIZE_DOC, MAX_LEN)
        toc(t1)
        torch.save(doc_embs, cache_path)
        print(f"[cache] saved -> {cache_path}")

    index, is_faiss = build_index(doc_embs)

    q_texts = [queries_map_pref[qid] for qid in qids_list]
    t2 = tic(f"Encoding queries (N={len(q_texts):,})…")
    q_embs  = encode_texts(mdl, tok, q_texts, BATCH_SIZE_Q, MAX_LEN)
    toc(t2)

    t3 = tic("Searching…")
    I, D = search_index(index, is_faiss, q_embs, topk=max(K_LIST))
    toc(t3)

    run = {}
    for i, qid in enumerate(qids_list):
        ids = I[i]; scores = D[i]
        run[qid] = [(doc_ids[j], float(scores[k])) for k, j in enumerate(ids)]

    m = eval_multi_k(run, qrels, K_LIST)
    print("Metrics:", fmt(m))
    return m

# BM25 (raw text, no prefixes)
token_re = re.compile(r"[a-zа-яё0-9]+", re.IGNORECASE)
def ru_tok(s: str): return token_re.findall(s.lower())

def bm25_run(doc_texts: List[str], doc_ids: List[str], qids: List[str], queries_map: Dict[str, str], topk: int):
    toks = []
    for t in tqdm(doc_texts, desc="BM25 tokenize docs", total=len(doc_texts)):
        toks.append(ru_tok(t))
    t0 = tic("Building BM25 index…")
    bm = BM25Okapi(toks)
    toc(t0)

    run = {}
    for qid in tqdm(qids, desc="BM25 search", total=len(qids)):
        q_tokens = ru_tok(queries_map[qid])
        scores = bm.get_scores(q_tokens)
        if topk >= len(doc_ids):
            idx = np.argsort(scores)[::-1]
        else:
            part = np.argpartition(scores, -topk)[-topk:]
            idx = part[np.argsort(scores[part])[::-1]]
        run[qid] = [(doc_ids[j], float(scores[j])) for j in idx]
    return run

results = {}

# 1) BM25 (plain)
print("\n>>> BM25 (plain)")
bm_run_plain = bm25_run(doc_texts_raw, doc_ids, qids_all, query_map_raw, topk=max(K_LIST))
results["bm25_plain"] = eval_multi_k(bm_run_plain, qrels, K_LIST)
print("BM25 (plain) ->", fmt(results["bm25_plain"]))

if qids_with_instr:
    print("\n>>> BM25 (q+instr)  [on queries with instructions only]")
    raw_instr_map = {qid: (queries[qid] + " " + instructions[qid]).strip() for qid in qids_with_instr}
    bm_run_instr = bm25_run(doc_texts_raw, doc_ids, qids_with_instr, raw_instr_map, topk=max(K_LIST))
    results["bm25_instr"] = eval_multi_k(bm_run_instr, qrels, K_LIST)
    print("BM25 (instr) ->", fmt(results["bm25_instr"]))

# 2) Baseline encoder (no adapters) — plain
results["base_plain"] = eval_model(BASE_MODEL, "Baseline (mE5-base) :: plain", query_map_pref_plain, qids_all)

# 2b) Baseline encoder — q+instr 
if qids_with_instr:
    results["base_instr"] = eval_model(BASE_MODEL, "Baseline (mE5-base) :: q+instr", query_map_pref_instr, qids_with_instr)

# 3) LoRA adapters
if os.path.isdir(FINETUNED_INST_DIR):
    results["ft_inst_plain"] = eval_model(FINETUNED_INST_DIR, "Finetuned (instruction-aware, LoRA) :: plain", query_map_pref_plain, qids_all)
    if qids_with_instr:
        results["ft_inst_instr"] = eval_model(FINETUNED_INST_DIR, "Finetuned (instruction-aware, LoRA) :: q+instr", query_map_pref_instr, qids_with_instr)
else:
    print(f"[skip] {FINETUNED_INST_DIR} not found.")

if os.path.isdir(FINETUNED_QUERY_DIR):
    results["ft_query_plain"] = eval_model(FINETUNED_QUERY_DIR, "Finetuned (query-only, LoRA) :: plain", query_map_pref_plain, qids_all)
    if qids_with_instr:
        results["ft_query_instr"] = eval_model(FINETUNED_QUERY_DIR, "Finetuned (query-only, LoRA) :: q+instr", query_map_pref_instr, qids_with_instr)
else:
    print(f"[skip] {FINETUNED_QUERY_DIR} not found.")

# Summary
print("\n=== SUMMARY (Mr.TyDi-RU) ===")
for k, v in results.items():
    print(f"{k:14s} ->", fmt(v))

# p-MRR 
if qids_with_instr:
    print("\n=== p-MRR (instr − plain) on queries with instructions only ===")
    def safe_pmrr(tag_plain, tag_instr):
        if tag_plain in results and tag_instr in results:
            return pmrr(results[tag_plain], results[tag_instr], K_LIST)
        return None

    pmrr_base = safe_pmrr("base_plain", "base_instr")
    if pmrr_base:
        print("Baseline:", fmt(pmrr_base))

    pmrr_ft_inst = safe_pmrr("ft_inst_plain", "ft_inst_instr")
    if pmrr_ft_inst:
        print("FT-Inst:", fmt(pmrr_ft_inst))

    pmrr_ft_q = safe_pmrr("ft_query_plain", "ft_query_instr")
    if pmrr_ft_q:
        print("FT-Query:", fmt(pmrr_ft_q))

    pmrr_bm25 = safe_pmrr("bm25_plain", "bm25_instr")
    if pmrr_bm25:
        print("BM25:   ", fmt(pmrr_bm25))
else:
    print("\n[p-MRR] No instructions.jsonl found or no overlap with qrels — skipping p-MRR.")


Device: cuda | DOC_CAP=20000 | FAST_Q_CAP=None
Loading dataset files…

Loading corpus: 100%|██████████| 5.84G/5.84G [01:19<00:00, 73.4MB/s]
Loading queries: 100%|██████████| 113k/113k [00:00<00:00, 29.2MB/s]
Loading qrels: 100%|██████████| 19.0k/19.0k [00:00<00:00, 8.20MB/s]
Loading instructions: 100%|██████████| 268k/268k [00:00<00:00, 56.0MB/s]


 Loaded (docs=9,597,504, queries=995, qrels_q=995, instr_q=995) in 79.6s.
Eval set -> docs: 20,000 (positives kept: 1,100, random negatives added: 18900) | queries: 995

>>> BM25 (plain)


BM25 tokenize docs: 100%|██████████| 20000/20000 [00:00<00:00, 20717.27it/s]

Building BM25 index…

...

 done in 1.1s.
Searching…

 done in 0.1s.
Metrics: {'N': 995, 'Hit@1': 0.7518, 'Recall@1': 0.7518, 'MRR@1': 0.7518, 'nDCG@1': 0.7518, 'Recall@5': 0.8975, 'MRR@5': 0.8131, 'nDCG@5': 0.8201, 'Recall@10': 0.9286, 'MRR@10': 0.8171, 'nDCG@10': 0.8308}

=== Finetuned (query-only, LoRA) :: q+instr ===
Loading model/tokenizer… done in 1.8s.
[cache] doc embs loaded: torch.Size([20000, 768])
Encoding queries (N=995)…

Encode@cuda:0: 100%|██████████| 16/16 [00:01<00:00,  8.39it/s]

 done in 1.9s.
Searching…

 done in 0.1s.
Metrics: {'N': 995, 'Hit@1': 0.7427, 'Recall@1': 0.7427, 'MRR@1': 0.7427, 'nDCG@1': 0.7427, 'Recall@5': 0.8915, 'MRR@5': 0.8046, 'nDCG@5': 0.8118, 'Recall@10': 0.9166, 'MRR@10': 0.8081, 'nDCG@10': 0.8217}

=== SUMMARY (Mr.TyDi-RU) ===
bm25_plain     -> {'N': 995, 'Hit@1': 0.4171, 'Recall@1': 0.4171, 'MRR@1': 0.4171, 'nDCG@1': 0.4171, 'Recall@5': 0.5548, 'MRR@5': 0.4713, 'nDCG@5': 0.4676, 'Recall@10': 0.599, 'MRR@10': 0.477, 'nDCG@10': 0.4813}
bm25_instr     -> {'N': 995, 'Hit@1': 0.3789, 'Recall@1': 0.3789, 'MRR@1': 0.3789, 'nDCG@1': 0.3789, 'Recall@5': 0.5025, 'MRR@5': 0.4265, 'nDCG@5': 0.4215, 'Recall@10': 0.5457, 'MRR@10': 0.4322, 'nDCG@10': 0.4346}
base_plain     -> {'N': 995, 'Hit@1': 0.808, 'Recall@1': 0.808, 'MRR@1': 0.808, 'nDCG@1': 0.808, 'Recall@5': 0.9357, 'MRR@5': 0.862, 'nDCG@5': 0.8686, 'Recall@10': 0.9588, 'MRR@10': 0.8653, 'nDCG@10': 0.8782}
base_instr     -> {'N': 995, 'Hit@1': 0.7779, 'Recall@1': 0.7779, 'MRR@1': 0.7779, 'nDCG@1': 0.7779, 'Recall@5': 0.

In [17]:
# RuBQ_2.0 
from pathlib import Path
import requests, zipfile, io, shutil, os

TARGET = Path("/kaggle/working/RuBQ_2.0")

def ensure_rubq_2_0(target: Path = TARGET) -> Path:
    target.mkdir(parents=True, exist_ok=True)
    has_any = any(target.rglob("*.json")) or any(target.rglob("*.jsonl"))
    if has_any:
        print(f"[ok] RuBQ_2.0 already present at: {target}")
        return target

    url = "https://codeload.github.com/vladislavneon/RuBQ/zip/refs/heads/master"
    print("[fetch] downloading RuBQ repo zip…")
    r = requests.get(url, timeout=90)
    r.raise_for_status()

    with zipfile.ZipFile(io.BytesIO(r.content)) as zf:
        top_levels = {name.split("/")[0] for name in zf.namelist() if "/" in name}
        if not top_levels:
            raise RuntimeError("Unexpected zip layout (no top-level folder).")
        root_dir = sorted(top_levels)[0]  # e.g., 'RuBQ-master'
        inner_prefix = f"{root_dir}/RuBQ_2.0/"
        members = [m for m in zf.namelist() if m.startswith(inner_prefix)]
        if not members:
            raise RuntimeError("Couldn’t find 'RuBQ_2.0/' inside the repo zip.")
        extract_base = Path("/kaggle/working/_rubq_tmp")
        if extract_base.exists():
            shutil.rmtree(extract_base, ignore_errors=True)
        extract_base.mkdir(parents=True, exist_ok=True)
        zf.extractall(extract_base)

    src = extract_base / inner_prefix
    if target.exists():
        shutil.rmtree(target, ignore_errors=True)
    shutil.move(str(src), str(target))
    shutil.rmtree(extract_base, ignore_errors=True)

    print(f"[ok] RuBQ_2.0 ready at: {target}")
    return target

RUBQ_DIR = str(ensure_rubq_2_0())

json_count  = sum(1 for _ in Path(RUBQ_DIR).rglob("*.json"))
jsonl_count = sum(1 for _ in Path(RUBQ_DIR).rglob("*.jsonl"))
print(f"[scan] Found JSON: {json_count:,} | JSONL: {jsonl_count:,}")
print("Set RUBQ_DIR =", RUBQ_DIR)


[fetch] downloading RuBQ repo zip…
[ok] RuBQ_2.0 ready at: /kaggle/working/RuBQ_2.0
[scan] Found JSON: 3 | JSONL: 0
Set RUBQ_DIR = /kaggle/working/RuBQ_2.0


In [22]:
# RuBQ_2.0 -> Minimal IR Eval (BM25 + Base + LoRA)

import os, json, re, math, random, hashlib
from collections import defaultdict, Counter
from typing import Dict, List, Tuple

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

import numpy as np
from tqdm import tqdm
from rank_bm25 import BM25Okapi

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel

DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE_DOC = 64
BATCH_SIZE_Q   = 64
MAX_LEN        = 256
K_LIST         = [1, 5, 10]
SEED           = 42

DOC_CAP        = None     
FAST_Q_CAP     = None      

USE_PREFIXES   = True

BASE_MODEL          = "intfloat/multilingual-e5-base"
FINETUNED_INST_DIR  = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"      # instruction-aware
FINETUNED_QUERY_DIR = "/kaggle/working/mE5_lora8bit_queryonly_v1/best"    # query-only

RUBQ_DIR = "/kaggle/working/RuBQ_2.0"

ROOT_OUT  = "/kaggle/working/rubq_ir_eval"
EMB_CACHE = os.path.join(ROOT_OUT, "emb_cache")
os.makedirs(ROOT_OUT, exist_ok=True)
os.makedirs(EMB_CACHE, exist_ok=True)

random.seed(SEED); np.random.seed(SEED)
print(f"Device: {DEVICE} | DOC_CAP={DOC_CAP} | FAST_Q_CAP={FAST_Q_CAP}")

def add_prefix(s: str, kind: str) -> str:
    if not USE_PREFIXES: return s
    return (("query: " if kind == "query" else "passage: ") + s).strip()

def fmt(d): return {k:(round(v,4) if isinstance(v,float) else v) for k,v in d.items()}

def signature(doc_ids: List[str], name: str):
    head="||".join(doc_ids[:50]); tail="||".join(doc_ids[-50:]) if len(doc_ids)>50 else ""
    return hashlib.md5((head+"|"+tail+f"|{len(doc_ids)}").encode()).hexdigest()[:8] + f"__L{MAX_LEN}__{name.replace('/','_')}"

# Parse RuBQ JSONs
def first_nonempty(d: dict, keys: List[str]):
    for k in keys:
        v = d.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()
    return None

def extract_evidence_texts(item) -> List[str]:
    texts = []
    for k in ["evidence", "evidences"]:
        if k in item:
            ev = item[k]
            if isinstance(ev, str) and ev.strip():
                texts.append(ev.strip())
            elif isinstance(ev, list):
                for e in ev:
                    if isinstance(e, str) and e.strip():
                        texts.append(e.strip())
                    elif isinstance(e, dict):
                        t = first_nonempty(e, ["text", "snippet", "abstract", "passage", "content"])
                        if t: texts.append(t)
    return texts

def extract_answer_texts(item) -> List[str]:
    out = []
    if "answers" in item and isinstance(item["answers"], list):
        for a in item["answers"]:
            if isinstance(a, str) and a.strip():
                out.append(a.strip())
            elif isinstance(a, dict):
                t = first_nonempty(a, ["label", "text", "name", "description"])
                if t: out.append(t)
    if "answer" in item and isinstance(item["answer"], dict):
        t = first_nonempty(item["answer"], ["label", "text", "name", "description"])
        if t: out.append(t)
    for k in ["label", "description"]:
        if isinstance(item.get(k), str) and item[k].strip():
            out.append(item[k].strip())
    uniq = []
    seen = set()
    for s in out:
        s2 = s.strip()
        if s2 and s2 not in seen:
            seen.add(s2); uniq.append(s2)
    return uniq

def scan_jsons(root: str) -> List[dict]:
    all_items = []
    if not os.path.isdir(root):
        return []
    for dirpath, _, files in os.walk(root):
        for fn in files:
            if not fn.lower().endswith(".json"): continue
            fp = os.path.join(dirpath, fn)
            try:
                data = json.load(open(fp, "r", encoding="utf-8"))
                if isinstance(data, list):
                    all_items.extend(data)
                elif isinstance(data, dict):
                    for v in data.values():
                        if isinstance(v, list):
                            all_items.extend(v)
            except Exception as e:
                print(f"[warn] bad JSON: {fp} ({type(e).__name__})")
    return all_items

print("\n[prep] Scanning RuBQ JSON…")
raw_items = scan_jsons(RUBQ_DIR)
print(f"[prep] Loaded raw entries: {len(raw_items)}")

corpus: Dict[str, str] = {}
queries: Dict[str, str] = {}
qrels: Dict[str, Dict[str, int]] = defaultdict(dict)

def add_doc_get_id(text: str, text2id: Dict[str, str], prefix="D") -> str:
    h = hashlib.md5(text.encode("utf-8")).hexdigest()[:16]
    did = f"{prefix}_{h}"
    text2id[did] = text
    return did

q_missing = 0
has_evidence = 0
used_fallback = 0

for i, item in enumerate(tqdm(raw_items, desc="Parsing rows")):
    q = first_nonempty(item, ["question", "question_text", "question_ru", "text"])
    if not q:
        q_missing += 1
        continue
    qid = f"Q_{i:08d}"
    queries[qid] = q

    ev_txts = extract_evidence_texts(item)
    if ev_txts:
        has_evidence += 1
        for t in ev_txts:
            did = add_doc_get_id(t, corpus, prefix="E")
            qrels[qid][did] = 1
        continue

    ans_txts = extract_answer_texts(item)
    if ans_txts:
        used_fallback += 1
        for t in ans_txts:
            did = add_doc_get_id(t, corpus, prefix="A")
            qrels[qid][did] = 1

print(f"[prep] Done. Docs={len(corpus)} | Queries={len(queries)} | QrelsQ={len(qrels)}")
print(f"[prep] Items without question: {q_missing}")
print(f"[prep] Items with 'evidence*': {has_evidence} | using fallback(Label/Desc): {used_fallback}")

if len(corpus) == 0 or len(queries) == 0 or len(qrels) == 0:
    print("\n[STOP] Parsed dataset is too small/empty for IR.")
    print(" - Ensure your RuBQ JSONs include 'evidence/evidences' or answer labels/descriptions.")
    raise SystemExit(0)

doc_ids = list(corpus.keys())
if DOC_CAP and DOC_CAP < len(doc_ids):
    rng = random.Random(SEED); rng.shuffle(doc_ids); doc_ids = doc_ids[:DOC_CAP]
corpus = {d: corpus[d] for d in doc_ids}

qids_all = [qid for qid, rels in qrels.items() if any(d in corpus for d in rels)]
if FAST_Q_CAP and FAST_Q_CAP < len(qids_all):
    rng = random.Random(SEED); rng.shuffle(qids_all); qids_all = qids_all[:FAST_Q_CAP]

print(f"\n[eval] Final set -> docs={len(doc_ids)} | queries={len(qids_all)}")

# Metrics
def eval_multi_k(run, qrels_map, ks):
    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks:
        stats.update({
            f"Recall@{k}": 0.0,
            f"MRR@{k}":    0.0,
            f"nDCG@{k}":   0.0,
            f"MAP@{k}":    0.0,   
        })

    n = 0
    for qid, ranking in run.items():
        if qid not in qrels_map:
            continue
        n += 1
        rels = qrels_map[qid]                 
        pos_ranks = [r for r, (did, _) in enumerate(ranking, start=1) if did in rels]
        pos_ranks.sort()

        # Hit@1
        if pos_ranks and pos_ranks[0] == 1:
            stats["Hit@1"] += 1.0

        m = len(rels)                         

        for k in ks:
            stats[f"Recall@{k}"] += float(any(r <= k for r in pos_ranks))

            stats[f"MRR@{k}"] += (1.0 / pos_ranks[0]) if (pos_ranks and pos_ranks[0] <= k) else 0.0

            dcg  = sum(1.0 / math.log2(r + 1) for r in pos_ranks if r <= k)
            idcg = sum(1.0 / math.log2(r + 1) for r in range(1, min(m, k) + 1))
            stats[f"nDCG@{k}"] += (dcg / idcg) if idcg > 0 else 0.0

            denom = min(k, m)
            if denom > 0:
                ap_k = 0.0
                rel_found_so_far = 0
                j = 0
                for r in pos_ranks:
                    if r > k:
                        break
                    j += 1                    
                    rel_found_so_far = j
                    prec_at_r = rel_found_so_far / r
                    ap_k += prec_at_r
                stats[f"MAP@{k}"] += ap_k / denom
    if n == 0:
        return stats

    stats["N"] = n
    for key in ["Hit@1"] + \
               [f"Recall@{x}" for x in ks] + \
               [f"MRR@{x}" for x in ks] + \
               [f"nDCG@{x}" for x in ks] + \
               [f"MAP@{x}" for x in ks]:
        stats[key] /= n
    return stats

# BM25 
token_re = re.compile(r"[a-zа-яё0-9]+", re.IGNORECASE)
def ru_tok(s: str): return token_re.findall(s.lower())

def bm25_run(doc_texts: List[str], doc_ids: List[str], qids: List[str], qmap: Dict[str,str], topk: int):
    print("\n[BM25] Building index over docs…")
    bm = BM25Okapi([ru_tok(t) for t in tqdm(doc_texts, desc="BM25 build")])
    run={}
    for qid in tqdm(qids, desc="BM25 search"):
        toks = ru_tok(qmap[qid])
        scores = bm.get_scores(toks)
        if topk >= len(doc_ids): idx = np.argsort(scores)[::-1]
        else:
            part = np.argpartition(scores, -topk)[-topk:]
            idx = part[np.argsort(scores[part])[::-1]]
        run[qid]=[(doc_ids[j], float(scores[j])) for j in idx]
    return run

def mean_pool(last_hidden, attention_mask):
    m = attention_mask.unsqueeze(-1).expand_as(last_hidden).float()
    return (last_hidden * m).sum(1) / m.sum(1).clamp(min=1e-9)

def load_tok_model_any(model_or_adapter: str, base_model: str):
    is_adapter = os.path.isdir(model_or_adapter) and os.path.exists(os.path.join(model_or_adapter, "adapter_config.json"))
    tok_src = model_or_adapter if (os.path.isdir(model_or_adapter) and os.path.exists(os.path.join(model_or_adapter, "tokenizer.json"))) else base_model
    tok = AutoTokenizer.from_pretrained(tok_src, use_fast=True)
    if DEVICE=="cuda":
        base = AutoModel.from_pretrained(base_model, torch_dtype=torch.float16, low_cpu_mem_usage=True).to("cuda").eval()
    else:
        base = AutoModel.from_pretrained(base_model, low_cpu_mem_usage=True).to("cpu").eval()
    mdl = PeftModel.from_pretrained(base, model_or_adapter).eval() if is_adapter else base
    return tok, mdl

@torch.no_grad()
def encode_texts(mdl, tok, texts: List[str], bs: int, max_len: int, device: str):
    outs=[]
    for i in tqdm(range(0,len(texts),bs), desc=f"Encode@{device}"):
        batch=texts[i:i+bs]
        enc=tok(batch,padding=True,truncation=True,max_length=max_len,return_tensors="pt")
        for k in enc: enc[k]=enc[k].to(device)
        last=mdl(**enc).last_hidden_state
        pooled=mean_pool(last, enc["attention_mask"])
        outs.append(F.normalize(pooled,p=2,dim=1).cpu())
    return torch.cat(outs,0) if outs else torch.empty(0, mdl.config.hidden_size)

def build_index(doc_embs_cpu: torch.Tensor):
    try:
        import faiss
        index = faiss.IndexFlatIP(doc_embs_cpu.shape[1])
        index.add(doc_embs_cpu.numpy().astype("float32"))
        return index, True
    except Exception:
        return doc_embs_cpu, False

@torch.no_grad()
def search_index(index, is_faiss: bool, q_embs: torch.Tensor, topk: int):
    if is_faiss:
        import faiss
        D, I = index.search(q_embs.numpy().astype("float32"), topk)
        return I, D
    sims = torch.mm(q_embs, index.t())
    D, I = torch.topk(sims, k=min(topk, sims.size(1)), dim=1)
    return I.numpy(), D.numpy()

def eval_model(model_or_adapter: str, label: str, doc_texts_pref: List[str], doc_ids: List[str],
               qids: List[str], qmap_pref: Dict[str,str]):
    print(f"\n=== {label} ===")
    tok, mdl = load_tok_model_any(model_or_adapter, BASE_MODEL)
    dev = str(next(mdl.parameters()).device)
    sig = signature(doc_ids, os.path.basename(model_or_adapter) if os.path.isdir(model_or_adapter) else model_or_adapter)
    cache_p = os.path.join(EMB_CACHE, f"docs__{sig}.pt")
    if os.path.exists(cache_p):
        doc_embs = torch.load(cache_p, map_location="cpu")
        print(f"[cache] doc embs: {doc_embs.shape}")
    else:
        doc_embs = encode_texts(mdl, tok, doc_texts_pref, BATCH_SIZE_DOC, MAX_LEN, dev)
        torch.save(doc_embs, cache_p)
        print(f"[cache] saved doc embs: {doc_embs.shape}")
    index, is_faiss = build_index(doc_embs)
    q_texts = [qmap_pref[qid] for qid in qids]
    q_embs  = encode_texts(mdl, tok, q_texts, BATCH_SIZE_Q, MAX_LEN, dev)
    I, D = search_index(index, is_faiss, q_embs, topk=max(K_LIST))
    run={}
    for i,qid in enumerate(qids):
        ids = I[i]; scores = D[i]
        run[qid]=[(doc_ids[j], float(scores[k])) for k,j in enumerate(ids)]
    m = eval_multi_k(run, qrels, K_LIST)
    print(fmt(m))
    return m

doc_ids = list(corpus.keys())
doc_texts_raw  = [corpus[d] for d in doc_ids]
doc_texts_pref = [add_prefix(t, "passage") for t in doc_texts_raw]

qids = list(qids_all)
query_map_pref = {qid: add_prefix(queries[qid], "query") for qid in qids}

# 1) BM25 ----------------
raw_query_map = {qid: queries[qid] for qid in qids}
bm_run = bm25_run(doc_texts_raw, doc_ids, qids, raw_query_map, topk=max(K_LIST))
bm25_metrics = eval_multi_k(bm_run, qrels, K_LIST)
print("\nBM25 ->", fmt(bm25_metrics))

# 2) Baseline encoder ----------------
base_metrics = eval_model(BASE_MODEL, "Baseline (mE5-base)", doc_texts_pref, doc_ids, qids, query_map_pref)

# 3) Adapters
if os.path.isdir(FINETUNED_INST_DIR):
    inst_metrics = eval_model(FINETUNED_INST_DIR, "Finetuned (instruction-aware, LoRA)", doc_texts_pref, doc_ids, qids, query_map_pref)
else:
    inst_metrics = None

if os.path.isdir(FINETUNED_QUERY_DIR):
    qo_metrics = eval_model(FINETUNED_QUERY_DIR, "Finetuned (query-only, LoRA)", doc_texts_pref, doc_ids, qids, query_map_pref)
else:
    qo_metrics = None

print("\n=== SUMMARY (RuBQ-derived IR) ===")
print("BM25   ->", fmt(bm25_metrics))
print("BASE   ->", fmt(base_metrics))
if inst_metrics is not None: print("FT-inst->", fmt(inst_metrics))
if qo_metrics   is not None: print("FT-qonly->", fmt(qo_metrics))

Device: cuda | DOC_CAP=None | FAST_Q_CAP=None

[prep] Scanning RuBQ JSON…
[prep] Loaded raw entries: 59862


Parsing rows: 100%|██████████| 59862/59862 [00:00<00:00, 388857.46it/s]


[prep] Done. Docs=3139 | Queries=59862 | QrelsQ=2400
[prep] Items without question: 0
[prep] Items with 'evidence*': 0 | using fallback(Label/Desc): 2400

[eval] Final set -> docs=3139 | queries=2400

[BM25] Building index over docs…


BM25 search: 100%|██████████| 2400/2400 [00:07<00:00, 328.87it/s]



BM25 -> {'N': 2400, 'Hit@1': 0.0229, 'Recall@1': 0.0229, 'MRR@1': 0.0229, 'nDCG@1': 0.0229, 'MAP@1': 0.0229, 'Recall@5': 0.0396, 'MRR@5': 0.029, 'nDCG@5': 0.0246, 'MAP@5': 0.0207, 'Recall@10': 0.0425, 'MRR@10': 0.0294, 'nDCG@10': 0.025, 'MAP@10': 0.0206}

=== Baseline (mE5-base) ===
[cache] doc embs: torch.Size([3139, 768])


...

{'N': 2400, 'Hit@1': 0.0758, 'Recall@1': 0.0758, 'MRR@1': 0.0758, 'nDCG@1': 0.0758, 'MAP@1': 0.0758, 'Recall@5': 0.1787, 'MRR@5': 0.1135, 'nDCG@5': 0.1117, 'MAP@5': 0.0925, 'Recall@10': 0.2308, 'MRR@10': 0.1203, 'nDCG@10': 0.128, 'MAP@10': 0.0987}

=== Finetuned (instruction-aware, LoRA) ===
[cache] doc embs: torch.Size([3139, 768])


Encode@cuda:0: 100%|██████████| 38/38 [00:02<00:00, 16.30it/s]


{'N': 2400, 'Hit@1': 0.0692, 'Recall@1': 0.0692, 'MRR@1': 0.0692, 'nDCG@1': 0.0692, 'MAP@1': 0.0692, 'Recall@5': 0.1671, 'MRR@5': 0.1045, 'nDCG@5': 0.1023, 'MAP@5': 0.0841, 'Recall@10': 0.2175, 'MRR@10': 0.1113, 'nDCG@10': 0.118, 'MAP@10': 0.0902}

=== Finetuned (query-only, LoRA) ===
[cache] doc embs: torch.Size([3139, 768])


Encode@cuda:0: 100%|██████████| 38/38 [00:02<00:00, 16.49it/s]


{'N': 2400, 'Hit@1': 0.0779, 'Recall@1': 0.0779, 'MRR@1': 0.0779, 'nDCG@1': 0.0779, 'MAP@1': 0.0779, 'Recall@5': 0.1825, 'MRR@5': 0.1164, 'nDCG@5': 0.1139, 'MAP@5': 0.0944, 'Recall@10': 0.2267, 'MRR@10': 0.1221, 'nDCG@10': 0.1279, 'MAP@10': 0.0998}

=== SUMMARY (RuBQ-derived IR) ===
BM25   -> {'N': 2400, 'Hit@1': 0.0229, 'Recall@1': 0.0229, 'MRR@1': 0.0229, 'nDCG@1': 0.0229, 'MAP@1': 0.0229, 'Recall@5': 0.0396, 'MRR@5': 0.029, 'nDCG@5': 0.0246, 'MAP@5': 0.0207, 'Recall@10': 0.0425, 'MRR@10': 0.0294, 'nDCG@10': 0.025, 'MAP@10': 0.0206}
BASE   -> {'N': 2400, 'Hit@1': 0.0758, 'Recall@1': 0.0758, 'MRR@1': 0.0758, 'nDCG@1': 0.0758, 'MAP@1': 0.0758, 'Recall@5': 0.1787, 'MRR@5': 0.1135, 'nDCG@5': 0.1117, 'MAP@5': 0.0925, 'Recall@10': 0.2308, 'MRR@10': 0.1203, 'nDCG@10': 0.128, 'MAP@10': 0.0987}
FT-inst-> {'N': 2400, 'Hit@1': 0.0692, 'Recall@1': 0.0692, 'MRR@1': 0.0692, 'nDCG@1': 0.0692, 'MAP@1': 0.0692, 'Recall@5': 0.1671, 'MRR@5': 0.1045, 'nDCG@5': 0.1023, 'MAP@5': 0.0841, 'Recall@10': 0.217

In [23]:
import os, requests

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")  

def ensure_lang_file(lang: str, out_dir: str) -> str:
    os.makedirs(out_dir, exist_ok=True)
    fp = os.path.join(out_dir, f"{lang}.json")
    if os.path.exists(fp):
        return fp

    url = f"https://raw.githubusercontent.com/google-research-datasets/lareqa/master/xquad-r/{lang}.json"
    print(f"[fetch] downloading {lang}.json …")
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    with open(fp, "wb") as f:
        f.write(r.content)
    print(f"[fetch] saved -> {fp}")
    return fp

LAREQA_DIR = "/kaggle/working/lareqa_xquad_r"  
ru_path = ensure_lang_file("ru", LAREQA_DIR)
en_path = ensure_lang_file("en", LAREQA_DIR)
print("Files ready:", ru_path, en_path)

[fetch] downloading ru.json …
[fetch] saved -> /kaggle/working/lareqa_xquad_r/ru.json
[fetch] downloading en.json …
[fetch] saved -> /kaggle/working/lareqa_xquad_r/en.json
Files ready: /kaggle/working/lareqa_xquad_r/ru.json /kaggle/working/lareqa_xquad_r/en.json


In [24]:
# LAReQA (XQuAD-R) RU↔EN — BM25 + Base + LoRA adapters

import os, io, re, json, math, time, random, hashlib, requests
from collections import defaultdict
from typing import Dict, List, Tuple

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F

from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel

DEVICE           = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE_DOC   = 64
BATCH_SIZE_Q     = 64
MAX_LEN          = 256
K_LIST           = [1, 5, 10]
SEED             = 42

DOC_CAP_TARGET   = None   
FAST_Q_CAP       = None  
USE_PREFIXES     = True   

FINETUNED_INST_DIR  = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"
FINETUNED_QUERY_DIR = "/kaggle/working/mE5_lora8bit_queryonly_v1/best"
BASE_MODEL          = "intfloat/multilingual-e5-base"

LAREQA_DIR = "/kaggle/working/lareqa_xquad_r"   
INSTR_JSONL = os.path.join(LAREQA_DIR, "instructions.jsonl")  
os.makedirs(LAREQA_DIR, exist_ok=True)

def tic(msg): print(f"\n{msg}"); return time.time()
def toc(t0, prefix="Elapsed"): print(f"{prefix}: {time.time()-t0:.1f}s")

random.seed(SEED); np.random.seed(SEED)
print(f"Device: {DEVICE} | DOC_CAP_TARGET={DOC_CAP_TARGET} | FAST_Q_CAP={FAST_Q_CAP}")

RAW_URL = "https://raw.githubusercontent.com/google-research-datasets/lareqa/master/dataset/xquad-r/{lang}.json"

def ensure_lang_file(lang: str, out_dir: str) -> str:
    fp = os.path.join(out_dir, f"{lang}.json")
    if os.path.exists(fp):
        return fp
    try:
        r = requests.get(RAW_URL.format(lang=lang), timeout=30)
        r.raise_for_status()
        with open(fp, "w", encoding="utf-8") as f:
            f.write(r.text)
        return fp
    except Exception as e:
        print(f"[warn] Could not download {lang}.json automatically: {e}")
        print("       Please upload en.json and ru.json into", out_dir)
        return fp  

ru_path = ensure_lang_file("ru", LAREQA_DIR)
en_path = ensure_lang_file("en", LAREQA_DIR)
if not (os.path.exists(ru_path) and os.path.exists(en_path)):
    raise FileNotFoundError("Missing ru.json or en.json in LAREQA_DIR. Upload them and re-run.")

def split_sentences(context: str, sent_field) -> List[str]:
    if isinstance(sent_field, list) and len(sent_field) > 0 and isinstance(sent_field[0], list):
        sents = []
        for start, end in sent_field:
            try:
                s = context[start:end].strip()
                if s: sents.append(s)
            except Exception:
                pass
        return sents
    if isinstance(sent_field, list) and sent_field and isinstance(sent_field[0], str):
        return [s.strip() for s in sent_field if str(s).strip()]
    return re.split(r'\s*(?<=\.|\?|!)\s+', context.strip())

def load_xquadr(path: str):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    qa_map = {}     
    para_sents = []
    for art in tqdm(data.get("data", []), desc=f"Parse {os.path.basename(path)}"):
        for par in art.get("paragraphs", []):
            context = par.get("context", "") or ""
            sents   = split_sentences(context, par.get("sentences"))
            base = len(para_sents)
            para_sents.extend(sents)
            for qa in par.get("qas", []):
                qid   = qa.get("id") or qa.get("qid") or ""
                qtext = qa.get("question", "").strip()
                answers = [a.get("text", "").strip() for a in qa.get("answers", []) if str(a.get("text","")).strip()]
                qa_map[qid] = {'question': qtext, 'answers': answers}
    return para_sents, qa_map

t0 = tic("[load] Reading ru.json & en.json…")
ru_sents, ru_qas = load_xquadr(ru_path)
en_sents, en_qas = load_xquadr(en_path)
toc(t0)

print(f"Sents: RU={len(ru_sents):,} | EN={len(en_sents):,}")
print(f"QAs:   RU={len(ru_qas):,} | EN={len(en_qas):,}")

def normalize(s): return re.sub(r"\s+", " ", s.strip().lower())

def build_corpus_and_qrels(q_lang: str, cand_lang: str,
                           qas_src: Dict[str,dict], answers_tgt: Dict[str,dict],
                           cand_sents: List[str],
                           doc_cap_target: int = DOC_CAP_TARGET,
                           fast_q_cap: int = FAST_Q_CAP):
    doc_ids = [f"{cand_lang}-s{ix}" for ix in range(len(cand_sents))]
    if doc_cap_target and doc_cap_target < len(doc_ids):
        rng = random.Random(SEED)
        order = list(range(len(doc_ids))); rng.shuffle(order)
        keep = sorted(order[:doc_cap_target])
        cand_sents = [cand_sents[i] for i in keep]
        doc_ids    = [doc_ids[i]    for i in keep]

    qids_all = sorted(set(qas_src.keys()) & set(answers_tgt.keys()))
    if fast_q_cap and fast_q_cap < len(qids_all):
        rng = random.Random(SEED); rng.shuffle(qids_all)
        qids_all = sorted(qids_all[:fast_q_cap])

    q_texts = {qid: qas_src[qid]['question'] for qid in qids_all}

    cand_norm = [normalize(x) for x in cand_sents]
    qrels = defaultdict(dict)
    for qid in tqdm(qids_all, desc=f"qrels {q_lang}->{cand_lang}"):
        golds = [normalize(a) for a in answers_tgt[qid].get('answers', []) if a.strip()]
        if not golds: 
            continue
        for si, s in enumerate(cand_norm):
            if any(g and g in s for g in golds):
                qrels[qid][doc_ids[si]] = 1
    qids = [qid for qid in qids_all if qrels.get(qid)]
    qrels = {qid: qrels[qid] for qid in qids}
    return doc_ids, cand_sents, qids, q_texts, qrels

# RU->EN
doc_ids_en, docs_en, qids_ru2en, q_ru_map, qrels_ru2en = build_corpus_and_qrels(
    "ru", "en", qas_src=ru_qas, answers_tgt=en_qas, cand_sents=en_sents,
    doc_cap_target=DOC_CAP_TARGET, fast_q_cap=FAST_Q_CAP
)
print(f"[RU→EN] candidates={len(doc_ids_en):,} | queries={len(qids_ru2en):,} | qrels_q={len(qrels_ru2en):,}")

# EN->RU
doc_ids_ru, docs_ru, qids_en2ru, q_en_map, qrels_en2ru = build_corpus_and_qrels(
    "en", "ru", qas_src=en_qas, answers_tgt=ru_qas, cand_sents=ru_sents,
    doc_cap_target=DOC_CAP_TARGET, fast_q_cap=FAST_Q_CAP
)
print(f"[EN→RU] candidates={len(doc_ids_ru):,} | queries={len(qids_en2ru):,} | qrels_q={len(qrels_en2ru):,}")

def load_instructions(jsonl_path: str) -> Dict[str, str]:
    """
    Accepts flexible fields per line:
      {"id": "QID", "instruction": "..."}  OR
      {"qid": "QID", "inst": "..."}        OR
      {"qid": "...", "instruction": "..."} etc.
    """
    m = {}
    if not os.path.exists(jsonl_path):
        return m
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            try:
                j = json.loads(line)
            except:
                continue
            qid = str(j.get("qid") or j.get("id") or "").strip()
            inst = str(j.get("instruction") or j.get("inst") or "").strip()
            if qid and inst:
                m[qid] = inst
    return m

instr_map = load_instructions(INSTR_JSONL)
if instr_map:
    print(f"[instr] Loaded instructions for {len(instr_map):,} qids from {INSTR_JSONL}")
else:
    print("[instr] No instructions.jsonl found or empty — p-MRR will be skipped.")

def add_prefix(s: str, kind: str) -> str:
    if not USE_PREFIXES: return s
    return (("query: " if kind=="query" else "passage: ") + s).strip()

def fmt(d):
    return {k:(round(v,4) if isinstance(v,float) else v) for k,v in d.items()}

def mean_pool(last_hidden_state, attention_mask):
    m = attention_mask.unsqueeze(-1).expand_as(last_hidden_state).float()
    return (last_hidden_state * m).sum(dim=1) / m.sum(dim=1).clamp(min=1e-9)

def eval_multi_k(run, qrels: Dict[str, Dict[str,int]], ks: List[int]):
    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks:
        stats.update({f"Recall@{k}":0.0, f"MRR@{k}":0.0, f"nDCG@{k}":0.0, f"MAP@{k}":0.0})
    n=0
    for qid, ranking in run.items():
        if qid not in qrels: continue
        n += 1
        rels = qrels[qid]
        pos = [r for r,(did,_) in enumerate(ranking, start=1) if did in rels]
        pos.sort()

        if pos and pos[0]==1:
            stats["Hit@1"] += 1.0

        m_rel = len(rels)
        for k in ks:
            stats[f"Recall@{k}"] += float(any(r<=k for r in pos))
            stats[f"MRR@{k}"]    += (1.0/pos[0]) if (pos and pos[0]<=k) else 0.0
            dcg  = sum(1.0/math.log2(r+1) for r in pos if r<=k)
            idcg = sum(1.0/math.log2(r+1) for r in range(1, min(m_rel,k)+1))
            stats[f"nDCG@{k}"] += (dcg/idcg) if idcg>0 else 0.0

            hits=0; ap_sum=0.0
            for r,(did,_) in enumerate(ranking[:k], start=1):
                if did in rels:
                    hits += 1
                    ap_sum += hits / r
            denom = float(min(m_rel, k)) if m_rel>0 else 1.0
            stats[f"MAP@{k}"] += (ap_sum/denom) if denom>0 else 0.0

    if n==0: return stats
    stats["N"]=n
    for k in ["Hit@1"]+[f"Recall@{x}" for x in ks]+[f"MRR@{x}" for x in ks]+[f"nDCG@{x}" for x in ks]+[f"MAP@{x}" for x in ks]:
        stats[k]/=n
    return stats

# BM25
token_re = re.compile(r"[a-z\u0400-\u04FF0-9]+", re.IGNORECASE)
def tok(s): return token_re.findall(s.lower())

def bm25_eval(doc_texts: List[str], doc_ids: List[str], qids: List[str], qmap: Dict[str,str], qrels: Dict[str,Dict[str,int]], topk: int):
    bm = BM25Okapi([tok(t) for t in tqdm(doc_texts, desc="BM25 build")])
    run = {}
    for qid in tqdm(qids, desc="BM25 search"):
        scores = bm.get_scores(tok(qmap[qid]))
        if topk >= len(doc_ids):
            idx = np.argsort(scores)[::-1]
        else:
            part = np.argpartition(scores, -topk)[-topk:]
            idx  = part[np.argsort(scores[part])[::-1]]
        run[qid] = [(doc_ids[j], float(scores[j])) for j in idx]
    return eval_multi_k(run, qrels, K_LIST)

def load_tok_and_model(model_or_adapter: str, base_model: str):
    is_adapter_dir = os.path.isdir(model_or_adapter) and os.path.exists(os.path.join(model_or_adapter, "adapter_config.json"))
    tok_src = model_or_adapter if (os.path.isdir(model_or_adapter) and os.path.exists(os.path.join(model_or_adapter, "tokenizer.json"))) else base_model
    tok = AutoTokenizer.from_pretrained(tok_src, use_fast=True)
    if DEVICE=="cuda":
        base = AutoModel.from_pretrained(base_model, torch_dtype=torch.float16, low_cpu_mem_usage=True).to("cuda").eval()
    else:
        base = AutoModel.from_pretrained(base_model, low_cpu_mem_usage=True).to("cpu").eval()
    mdl = PeftModel.from_pretrained(base, model_or_adapter).eval() if is_adapter_dir else base
    return tok, mdl

@torch.no_grad()
def encode_many(mdl, tok, texts: List[str], bs: int, max_len: int, device: str):
    out = []
    for i in tqdm(range(0, len(texts), bs), desc=f"Encode@{device}"):
        batch = texts[i:i+bs]
        enc = tok(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        for k in enc: enc[k] = enc[k].to(device)
        last = mdl(**enc).last_hidden_state
        pooled = mean_pool(last, enc["attention_mask"])
        out.append(F.normalize(pooled, p=2, dim=1).cpu())
    return torch.cat(out, 0) if out else torch.empty(0, mdl.config.hidden_size)

def build_index(doc_embs_cpu: torch.Tensor):
    try:
        import faiss
        index = faiss.IndexFlatIP(doc_embs_cpu.shape[1])
        index.add(doc_embs_cpu.numpy().astype("float32"))
        return index, True
    except Exception:
        return doc_embs_cpu, False

@torch.no_grad()
def search(index, is_faiss: bool, q_embs: torch.Tensor, topk: int):
    if is_faiss:
        import faiss
        D, I = index.search(q_embs.numpy().astype("float32"), topk)
        return I, D
    sims = torch.mm(q_embs, index.t())
    D, I = torch.topk(sims, k=min(topk, sims.size(1)), dim=1)
    return I.numpy(), D.numpy()

def neural_eval(model_or_adapter: str, base_model: str,
                doc_texts: List[str], doc_ids: List[str],
                qids: List[str], qmap: Dict[str,str], qrels: Dict[str,Dict[str,int]]):
    print(f"\n[neural] {os.path.basename(model_or_adapter) if os.path.isdir(model_or_adapter) else model_or_adapter}")
    tok, mdl = load_tok_and_model(model_or_adapter, base_model)
    dev = next(mdl.parameters()).device

    doc_texts_pref = [add_prefix(t, "passage") for t in doc_texts]
    q_texts_pref   = [add_prefix(qmap[q], "query") for q in qids]

    t0 = tic("Encode docs…")
    d_embs = encode_many(mdl, tok, doc_texts_pref, BATCH_SIZE_DOC, MAX_LEN, str(dev))
    toc(t0)

    index, is_faiss = build_index(d_embs)

    t0 = tic("Encode queries…")
    q_embs = encode_many(mdl, tok, q_texts_pref, BATCH_SIZE_Q, MAX_LEN, str(dev))
    toc(t0)

    I, D = search(index, is_faiss, q_embs, topk=max(K_LIST))
    run = {}
    for i, q in enumerate(qids):
        ids = I[i]; scores = D[i]
        run[q] = [(doc_ids[j], float(scores[k])) for k, j in enumerate(ids)]
    m = eval_multi_k(run, qrels, K_LIST)
    print("Metrics:", fmt(m))
    return m

def p_mrr(stats_plain: dict, stats_instr: dict) -> dict:
    out = {}
    for k in list(stats_plain.keys()):
        if k.startswith("MRR@") and k in stats_instr:
            out[f"p{ k }"] = float(stats_instr[k] - stats_plain[k])
    return out

# Run: RU→EN & EN→RU ----------------
def run_one_direction(tag, doc_ids, docs, qids, qmap, qrels):
    print(f"\n=== Direction: {tag} ===")
    print(f"Docs={len(doc_ids):,} | Queries={len(qids):,} | QrelsQ={len(qrels):,}")

    # 1) BM25 (plain)
    bm_plain = bm25_eval(docs, doc_ids, qids, qmap, qrels, topk=max(K_LIST))
    print("BM25 (plain) ->", fmt(bm_plain))

    # 2) Base / LoRAs (plain)
    base_plain = neural_eval(BASE_MODEL, BASE_MODEL, docs, doc_ids, qids, qmap, qrels)
    ft_i_plain = neural_eval(FINETUNED_INST_DIR, BASE_MODEL, docs, doc_ids, qids, qmap, qrels) if os.path.isdir(FINETUNED_INST_DIR) else None
    ft_q_plain = neural_eval(FINETUNED_QUERY_DIR, BASE_MODEL, docs, doc_ids, qids, qmap, qrels) if os.path.isdir(FINETUNED_QUERY_DIR) else None

    # Optional p-MRR with instructions ---
    pmrr_block = {}
    if instr_map:
        qids_with_inst = [qid for qid in qids if qid in instr_map and instr_map[qid].strip()]
        if qids_with_inst:
            qmap_instr = {qid: (qmap[qid] + " " + instr_map[qid]).strip() for qid in qids_with_inst}
            qrels_sub  = {qid: qrels[qid] for qid in qids_with_inst}
            # BM25/BASE/LoRA with instruction-augmented queries
            bm_instr   = bm25_eval(docs, doc_ids, qids_with_inst, qmap_instr, qrels_sub, topk=max(K_LIST))
            base_instr = neural_eval(BASE_MODEL, BASE_MODEL, docs, doc_ids, qids_with_inst, qmap_instr, qrels_sub)
            ft_i_instr = neural_eval(FINETUNED_INST_DIR, BASE_MODEL, docs, doc_ids, qids_with_inst, qmap_instr, qrels_sub) if os.path.isdir(FINETUNED_INST_DIR) else None
            ft_q_instr = neural_eval(FINETUNED_QUERY_DIR, BASE_MODEL, docs, doc_ids, qids_with_inst, qmap_instr, qrels_sub) if os.path.isdir(FINETUNED_QUERY_DIR) else None

            pmrr_block["BM25"] = p_mrr({k:v for k,v in bm_plain.items() if k.startswith("MRR@")}, {k:v for k,v in bm_instr.items() if k.startswith("MRR@")})
            pmrr_block["BASE"] = p_mrr({k:v for k,v in base_plain.items() if k.startswith("MRR@")}, {k:v for k,v in base_instr.items() if k.startswith("MRR@")})
            if ft_i_plain and ft_i_instr:
                pmrr_block["FT_INST"] = p_mrr({k:v for k,v in ft_i_plain.items() if k.startswith("MRR@")}, {k:v for k,v in ft_i_instr.items() if k.startswith("MRR@")})
            if ft_q_plain and ft_q_instr:
                pmrr_block["FT_QUERY"] = p_mrr({k:v for k,v in ft_q_plain.items() if k.startswith("MRR@")}, {k:v for k,v in ft_q_instr.items() if k.startswith("MRR@")})
        else:
            print("[p-MRR] Instructions provided, but none overlap with current qids — skipping p-MRR.")
    else:
        print("[p-MRR] No instructions available — skipping p-MRR.")

    # Summary
    print("\n-- Summary:", tag)
    print("BM25:", fmt(bm_plain))
    print("BASE:", fmt(base_plain))
    if ft_i_plain: print("FT_INST:", fmt(ft_i_plain))
    if ft_q_plain: print("FT_QUERY:", fmt(ft_q_plain))
    if pmrr_block:
        print("\n=== p-MRR (instr − plain) ===")
        for name, d in pmrr_block.items():
            print(name + ":", fmt(d))

# RU→EN
run_one_direction("RU→EN", doc_ids_en, docs_en, qids_ru2en, q_ru_map, qrels_ru2en)

# EN→RU
run_one_direction("EN→RU", doc_ids_ru, docs_ru, qids_en2ru, q_en_map, qrels_en2ru)


Device: cuda | DOC_CAP_TARGET=None | FAST_Q_CAP=None

[load] Reading ru.json & en.json…


Parse en.json: 100%|██████████| 48/48 [00:00<00:00, 18991.28it/s]


Elapsed: 0.0s
Sents: RU=1,219 | EN=1,180
QAs:   RU=1,190 | EN=1,190


qrels ru->en: 100%|██████████| 1190/1190 [00:00<00:00, 1683.61it/s]


[RU→EN] candidates=1,180 | queries=1,187 | qrels_q=1,187


...

Elapsed: 1.2s
Metrics: {'N': 1188, 'Hit@1': 0.6995, 'Recall@1': 0.6995, 'MRR@1': 0.6995, 'nDCG@1': 0.6995, 'MAP@1': 0.6995, 'Recall@5': 0.915, 'MRR@5': 0.786, 'nDCG@5': 0.7346, 'MAP@5': 0.6829, 'Recall@10': 0.9537, 'MRR@10': 0.7913, 'nDCG@10': 0.7469, 'MAP@10': 0.6866}

[neural] best

Encode docs…


Encode@cuda:0: 100%|██████████| 20/20 [00:04<00:00,  4.49it/s]


Elapsed: 4.5s

Encode queries…


Encode@cuda:0: 100%|██████████| 19/19 [00:01<00:00, 15.99it/s]


Elapsed: 1.2s
Metrics: {'N': 1188, 'Hit@1': 0.7222, 'Recall@1': 0.7222, 'MRR@1': 0.7222, 'nDCG@1': 0.7222, 'MAP@1': 0.7222, 'Recall@5': 0.9285, 'MRR@5': 0.8044, 'nDCG@5': 0.7511, 'MAP@5': 0.7005, 'Recall@10': 0.9588, 'MRR@10': 0.8086, 'nDCG@10': 0.7602, 'MAP@10': 0.7027}
[p-MRR] No instructions available — skipping p-MRR.

-- Summary: EN→RU
BM25: {'N': 1188, 'Hit@1': 0.0581, 'Recall@1': 0.0581, 'MRR@1': 0.0581, 'nDCG@1': 0.0581, 'MAP@1': 0.0581, 'Recall@5': 0.1347, 'MRR@5': 0.0864, 'nDCG@5': 0.0853, 'MAP@5': 0.0719, 'Recall@10': 0.1709, 'MRR@10': 0.091, 'nDCG@10': 0.0959, 'MAP@10': 0.076}
BASE: {'N': 1188, 'Hit@1': 0.7449, 'Recall@1': 0.7449, 'MRR@1': 0.7449, 'nDCG@1': 0.7449, 'MAP@1': 0.7449, 'Recall@5': 0.9276, 'MRR@5': 0.8207, 'nDCG@5': 0.7613, 'MAP@5': 0.7149, 'Recall@10': 0.9596, 'MRR@10': 0.8251, 'nDCG@10': 0.7705, 'MAP@10': 0.7168}
FT_INST: {'N': 1188, 'Hit@1': 0.6995, 'Recall@1': 0.6995, 'MRR@1': 0.6995, 'nDCG@1': 0.6995, 'MAP@1': 0.6995, 'Recall@5': 0.915, 'MRR@5': 0.786, 'nDC

In [25]:
# eval_mfollowir_full_with_pMRR.py
# - Evaluates BASE and FINETUNED on mFollowIR (RU)
# - Tests plain and (q + instruction_og) against qrels_og
# - OOM-safe loading + caching doc embeddings
# - Adds p-MRR@k (paired uplift: instr MRR - plain MRR)

import os, math, random, hashlib
from collections import defaultdict
from typing import Dict, List, Tuple
import numpy as np
import torch
import torch.nn.functional as F
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE_DOC = 32
BATCH_SIZE_Q   = 64
MAX_LEN        = 256
K_LIST         = [1, 10, 20]
SEED           = 42
FAST_Q_CAP     = None
DOC_CAP        = None
FP16_ON_GPU    = True
CACHE_DIR      = "/kaggle/working/mfollowir_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

BASE_MODEL         = "intfloat/multilingual-e5-base"
FINETUNED_INST_DIR = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"  # LoRA adapters folder or full model dir

def add_prefix(s, kind): 
    return (("query: " if kind == "query" else "passage: ") + s).strip()

@torch.no_grad()
def encode_texts(model, tok, texts, batch_size, max_len, device):
    outs = []
    for i in tqdm(range(0, len(texts), batch_size), desc=f"Encoding@{device}", leave=False):
        T = texts[i:i+batch_size]
        enc = tok(T, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        for k in enc: enc[k] = enc[k].to(device)
        out  = model(**enc).last_hidden_state
        mask = enc["attention_mask"].unsqueeze(-1).expand_as(out).float()
        pooled = (out * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        outs.append(F.normalize(pooled, p=2, dim=1).cpu())
    return torch.cat(outs, 0) if outs else torch.empty(0)

def build_index(embs):
    import faiss
    idx = faiss.IndexFlatIP(embs.shape[1])
    idx.add(embs.numpy().astype("float32"))
    return idx

def search(idx, q_embs, topk):
    import faiss
    D, I = idx.search(q_embs.numpy().astype("float32"), topk)
    return I, D

def is_adapter_dir(path: str) -> bool:
    return (os.path.isdir(path) and os.path.exists(os.path.join(path, "adapter_config.json")))

def safe_load(model_or_adapter_path: str, base_model: str = BASE_MODEL):
    
    tok_src = model_or_adapter_path if (os.path.isdir(model_or_adapter_path) and os.path.exists(os.path.join(model_or_adapter_path, "tokenizer.json"))) else base_model
    tok = AutoTokenizer.from_pretrained(tok_src, use_fast=True)

    try:
        mdl_base = AutoModel.from_pretrained(
            base_model if is_adapter_dir(model_or_adapter_path) else model_or_adapter_path,
            torch_dtype=(torch.float16 if (DEVICE == "cuda" and FP16_ON_GPU) else None),
            low_cpu_mem_usage=True
        ).to(DEVICE if DEVICE == "cuda" else "cpu").eval()
    except RuntimeError:
        mdl_base = AutoModel.from_pretrained(
            base_model if is_adapter_dir(model_or_adapter_path) else model_or_adapter_path,
            low_cpu_mem_usage=True
        ).to("cpu").eval()

    if is_adapter_dir(model_or_adapter_path):
        mdl = PeftModel.from_pretrained(mdl_base, model_or_adapter_path).eval()
    else:
        mdl = mdl_base
    dev = "cuda" if next(mdl.parameters()).is_cuda else "cpu"
    return tok, mdl, dev

def eval_multi_k_and_perquery(run, qrels, ks) -> Tuple[Dict, Dict[int, Dict[str, float]]]:

    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks: stats.update({f"Recall@{k}":0.0, f"MRR@{k}":0.0, f"nDCG@{k}":0.0})
    per_mrr = {k:{} for k in ks}
    n = 0
    for qid, ranking in run.items():
        if qid not in qrels: continue
        n += 1
        rels = qrels[qid]
        pos = [r for r,(did,_) in enumerate(ranking, start=1) if did in rels and rels[did] > 0]
        pos.sort()
        if pos and pos[0] == 1: stats["Hit@1"] += 1.0
        for k in ks:
            mrr_q = (1.0 / pos[0]) if (pos and pos[0] <= k) else 0.0
            per_mrr[k][qid] = mrr_q
            stats[f"Recall@{k}"] += float(any(r <= k for r in pos))
            stats[f"MRR@{k}"]    += mrr_q
            dcg  = sum(1.0/math.log2(r+1) for r in pos if r <= k)
            m    = sum(1 for _ in rels if rels[_] > 0)
            idcg = sum(1.0/math.log2(r+1) for r in range(1, min(m, k)+1))
            stats[f"nDCG@{k}"] += (dcg/idcg) if idcg > 0 else 0.0
    if n == 0: return stats, per_mrr
    stats["N"] = n
    for k in ["Hit@1"]+[f"Recall@{x}" for x in ks]+[f"MRR@{x}" for x in ks]+[f"nDCG@{x}" for x in ks]:
        stats[k] /= n
    return stats, per_mrr

def compute_p_mrr(per_mrr_plain: Dict[str,float], per_mrr_instr: Dict[str,float]) -> float:

    qids = sorted(set(per_mrr_plain.keys()) & set(per_mrr_instr.keys()))
    if not qids: return 0.0
    diffs = [(per_mrr_instr[q] - per_mrr_plain[q]) for q in qids]
    return float(np.mean(diffs))

print("Loading mFollowIR (rus)…")
ds_q = load_dataset("jhu-clsp/mFollowIR-parquet", "queries-rus")["queries"]
ds_d = load_dataset("jhu-clsp/mFollowIR-parquet", "corpus-rus")["corpus"]
ds_og = load_dataset("jhu-clsp/mFollowIR-parquet", "qrels_og-rus")["test"]

doc_ids = [r["_id"] for r in ds_d]
doc_txt = [((r.get("title") or "") + " " + (r.get("text") or "")).strip() for r in ds_d]
if DOC_CAP:
    rng = random.Random(SEED)
    idx = list(range(len(doc_ids))); rng.shuffle(idx); keep = sorted(idx[:DOC_CAP])
    doc_ids = [doc_ids[i] for i in keep]
    doc_txt = [doc_txt[i] for i in keep]
doc_txt = [add_prefix(t, "passage") for t in doc_txt]
doc_set = set(doc_ids)

def to_qrels(rows):
    qd = defaultdict(dict)
    for r in rows:
        if r["corpus-id"] in doc_set:
            qd[r["query-id"]][r["corpus-id"]] = float(r["score"])
    return {k:v for k,v in qd.items() if v}

qrels_og = to_qrels(ds_og)

q_plain_map = {r["_id"]: add_prefix(r["text"].strip(), "query") for r in ds_q}
q_og_map    = {r["_id"]: add_prefix((r["text"] + " " + r["instruction_og"]).strip(), "query") for r in ds_q}
qids = [qid for qid in q_plain_map if qid in qrels_og]
if FAST_Q_CAP:
    rng = random.Random(SEED); rng.shuffle(qids); qids = qids[:FAST_Q_CAP]
print(f"Docs={len(doc_ids)}, Queries={len(qids)}")

def evaluate(model_or_adapter_path: str, label: str, qmap: Dict[str,str]):
    print(f"\n=== {label} ===")
    tok, mdl, dev = safe_load(model_or_adapter_path, BASE_MODEL)

    sig = hashlib.md5(("|".join(doc_ids[:50])+f"|{len(doc_ids)}").encode()).hexdigest()[:8]
    base_name = os.path.basename(model_or_adapter_path) if os.path.isdir(model_or_adapter_path) else model_or_adapter_path.replace("/","_")
    dcache = os.path.join(CACHE_DIR, f"docs__{base_name}__{sig}__L{MAX_LEN}.pt")
    if os.path.exists(dcache):
        doc_embs = torch.load(dcache, map_location="cpu")
    else:
        doc_embs = encode_texts(mdl, tok, doc_txt, BATCH_SIZE_DOC, MAX_LEN, dev)
        torch.save(doc_embs, dcache)

    index = build_index(doc_embs)

    q_texts = [qmap[qid] for qid in qids]
    q_embs  = encode_texts(mdl, tok, q_texts, BATCH_SIZE_Q, MAX_LEN, dev)

    I, D = search(index, q_embs, max(K_LIST))
    run = { qid: [(doc_ids[j], float(D[i][k])) for k, j in enumerate(I[i])] for i, qid in enumerate(qids) }

    metrics, per_mrr = eval_multi_k_and_perquery(run, qrels_og, K_LIST)
    print({k:(round(v,4) if isinstance(v,float) else v) for k,v in metrics.items()})
    return metrics, per_mrr

res = {}
per = {}

# Baseline (plain vs q+inst)
res["baseline_plain"], per["baseline_plain"] = evaluate(BASE_MODEL, "BASELINE :: plain vs qrels_og", q_plain_map)
res["baseline_instr"], per["baseline_instr"] = evaluate(BASE_MODEL, "BASELINE :: q+inst vs qrels_og", q_og_map)

# Finetuned (instruction-aware LoRA) 
res["ft_plain"],     per["ft_plain"]     = evaluate(FINETUNED_INST_DIR, "FINETUNED :: plain vs qrels_og", q_plain_map)
res["ft_instr"],     per["ft_instr"]     = evaluate(FINETUNED_INST_DIR, "FINETUNED :: q+inst vs qrels_og", q_og_map)

# p-MRR@k = mean_q ( MRR_instr@k(q) - MRR_plain@k(q) )
def p_mrr_block(tag_plain: str, tag_instr: str) -> Dict[str, float]:
    out = {}
    for k in K_LIST:
        pmrr = compute_p_mrr(per[tag_plain][k], per[tag_instr][k])
        out[f"pMRR@{k}"] = round(pmrr, 6)
    return out

p_baseline = p_mrr_block("baseline_plain", "baseline_instr")
p_ft       = p_mrr_block("ft_plain",       "ft_instr")

print("\n=== SUMMARY (mFollowIR-RU, qrels_og) ===")
for k,v in res.items():
    print(f"{k:16s} ->", {kk:(round(vv,4) if isinstance(vv,float) else vv) for kk,vv in v.items()})

print("\n=== p-MRR (instr − plain) ===")
print("Baseline:", p_baseline)
print("Finetuned:", p_ft)

Loading mFollowIR (rus)…


README.md: 0.00B [00:00, ?B/s]

queries-rus/queries.parquet:   0%|          | 0.00/46.6k [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/40 [00:00<?, ? examples/s]

corpus-rus/corpus.parquet:   0%|          | 0.00/100M [00:00<?, ?B/s]

...

Docs=39326, Queries=40

=== BASELINE :: plain vs qrels_og ===


{'N': 40, 'Hit@1': 0.475, 'Recall@1': 0.475, 'MRR@1': 0.475, 'nDCG@1': 0.475, 'Recall@10': 0.825, 'MRR@10': 0.597, 'nDCG@10': 0.3762, 'Recall@20': 0.875, 'MRR@20': 0.6004, 'nDCG@20': 0.3877}

=== BASELINE :: q+inst vs qrels_og ===


{'N': 40, 'Hit@1': 0.575, 'Recall@1': 0.575, 'MRR@1': 0.575, 'nDCG@1': 0.575, 'Recall@10': 0.775, 'MRR@10': 0.6308, 'nDCG@10': 0.389, 'Recall@20': 0.85, 'MRR@20': 0.6369, 'nDCG@20': 0.4076}

=== FINETUNED :: plain vs qrels_og ===


{'N': 40, 'Hit@1': 0.55, 'Recall@1': 0.55, 'MRR@1': 0.55, 'nDCG@1': 0.55, 'Recall@10': 0.8, 'MRR@10': 0.6184, 'nDCG@10': 0.4079, 'Recall@20': 0.875, 'MRR@20': 0.6232, 'nDCG@20': 0.4013}

=== FINETUNED :: q+inst vs qrels_og ===


{'N': 40, 'Hit@1': 0.575, 'Recall@1': 0.575, 'MRR@1': 0.575, 'nDCG@1': 0.575, 'Recall@10': 0.8, 'MRR@10': 0.6276, 'nDCG@10': 0.4282, 'Recall@20': 0.85, 'MRR@20': 0.6312, 'nDCG@20': 0.4364}

=== SUMMARY (mFollowIR-RU, qrels_og) ===
baseline_plain   -> {'N': 40, 'Hit@1': 0.475, 'Recall@1': 0.475, 'MRR@1': 0.475, 'nDCG@1': 0.475, 'Recall@10': 0.825, 'MRR@10': 0.597, 'nDCG@10': 0.3762, 'Recall@20': 0.875, 'MRR@20': 0.6004, 'nDCG@20': 0.3877}
baseline_instr   -> {'N': 40, 'Hit@1': 0.575, 'Recall@1': 0.575, 'MRR@1': 0.575, 'nDCG@1': 0.575, 'Recall@10': 0.775, 'MRR@10': 0.6308, 'nDCG@10': 0.389, 'Recall@20': 0.85, 'MRR@20': 0.6369, 'nDCG@20': 0.4076}
ft_plain         -> {'N': 40, 'Hit@1': 0.55, 'Recall@1': 0.55, 'MRR@1': 0.55, 'nDCG@1': 0.55, 'Recall@10': 0.8, 'MRR@10': 0.6184, 'nDCG@10': 0.4079, 'Recall@20': 0.875, 'MRR@20': 0.6232, 'nDCG@20': 0.4013}
ft_instr         -> {'N': 40, 'Hit@1': 0.575, 'Recall@1': 0.575, 'MRR@1': 0.575, 'nDCG@1': 0.575, 'Recall@10': 0.8, 'MRR@10': 0.6276, 'nDCG@10

In [26]:
# rus-XQuAD / rus-NFCorpus / WikiFacts-Articles — HF qrels evaluation

import os, io, gzip, glob, json, math, shutil, random, csv
from typing import Dict, List, Tuple
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import snapshot_download
from tqdm import tqdm

DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE    = 64
MAX_LEN       = 256
K_LIST        = [2, 5, 10]
SEED          = 42
random.seed(SEED); np.random.seed(SEED)

FINETUNED_INST_DIR  = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"  
FINETUNED_QUERY_DIR = "/kaggle/working/mE5_lora8bit_queryonly_v1/best"               
BASE_MODEL          = "intfloat/multilingual-e5-base"

FAST_Q_CAP   = 300     
DOC_CAP      = None    
USE_PREFIXES = False  

CACHE_DIR    = "/kaggle/working/rusbeir_eval_cache"
os.makedirs(CACHE_DIR, exist_ok=True)

OUT_ROOT     = "/kaggle/working/rusbeir_eval_new"
HF_CACHE     = "/kaggle/working/hf_cache"
os.makedirs(OUT_ROOT, exist_ok=True)
os.makedirs(HF_CACHE,  exist_ok=True)

print("Device:", DEVICE)

SETS = {
    "rus-xquad": {
        "data_repo":  "kaengreg/rus-xquad",
        "qrels_repo": "kaengreg/rus-xquad-qrels",
        "qrels_file": "dev.tsv",
    },
    "rus-nfcorpus": {
        "data_repo":  "kaengreg/rus-nfcorpus",
        "qrels_repo": "kaengreg/rus-nfcorpus-qrels",
        "qrels_file": "test.tsv",
    },
    "wikifacts-articles": {
        "data_repo":  "kaengreg/wikifacts-articles",
        "qrels_repo": "kaengreg/wikifacts-articles-qrels",
        "qrels_file": "dev.tsv",
    },
}

def _copy(src, dst):
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy2(src, dst)

def _maybe_gunzip(src, dst):
    if src.endswith(".gz"):
        with gzip.open(src, "rt", encoding="utf-8") as f_in, open(dst, "w", encoding="utf-8") as f_out:
            for line in f_in: f_out.write(line)
    else:
        _copy(src, dst)

def fetch_raw_dataset(name: str, data_repo: str) -> str:
    out_dir = os.path.join(OUT_ROOT, name)
    os.makedirs(out_dir, exist_ok=True)
    root = snapshot_download(repo_id=data_repo, repo_type="dataset",
                             allow_patterns=["corpus.jsonl","queries.jsonl","*.gz"],
                             local_dir=HF_CACHE, local_dir_use_symlinks=False)
    cand_corpus  = glob.glob(os.path.join(root, "**", "corpus.jsonl*"),  recursive=True)
    cand_queries = glob.glob(os.path.join(root, "**", "queries.jsonl*"), recursive=True)
    if not cand_corpus or not cand_queries:
        raise FileNotFoundError(f"Could not find corpus.jsonl/queries.jsonl in {data_repo}")
    _maybe_gunzip(sorted(cand_corpus)[-1],  os.path.join(out_dir, "corpus.jsonl"))
    _maybe_gunzip(sorted(cand_queries)[-1], os.path.join(out_dir, "queries.jsonl"))
    return out_dir

def fetch_qrels(name: str, qrels_repo: str, qrels_file: str) -> str:
    out_dir = os.path.join(OUT_ROOT, name)
    os.makedirs(out_dir, exist_ok=True)
    root = snapshot_download(repo_id=qrels_repo, repo_type="dataset",
                             allow_patterns=[qrels_file],
                             local_dir=HF_CACHE, local_dir_use_symlinks=False)
    src = glob.glob(os.path.join(root, "**", qrels_file), recursive=True)
    if not src:
        raise FileNotFoundError(f"{qrels_file} not found in {qrels_repo}")
    dst = os.path.join(out_dir, "qrels.tsv")
    _copy(sorted(src)[-1], dst)
    return dst

def smart_open(path):
    if path.endswith(".gz"):
        return io.TextIOWrapper(gzip.open(path, "rb"), encoding="utf-8")
    return open(path, "r", encoding="utf-8")

def load_corpus(path: str) -> Dict[str, str]:
    corpus = {}
    with smart_open(path) as f:
        for line in f:
            j = json.loads(line)
            cid   = str(j.get("_id") or j.get("id"))
            title = (j.get("title") or "").strip()
            text  = (j.get("text")  or "").strip()
            corpus[cid] = (title + " " + text).strip()
    return corpus

def load_queries(path: str) -> Dict[str, str]:
    queries = {}
    with smart_open(path) as f:
        for line in f:
            j = json.loads(line)
            qid  = str(j.get("_id") or j.get("id"))
            text = (j.get("text") or "").strip()
            queries[qid] = text
    return queries

def load_qrels_tsv(path: str) -> Dict[str, Dict[str, int]]:
    qrels = defaultdict(dict)
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        header = None
        qid_idx, did_idx, rel_idx = 0, 1, 2
        for i, row in enumerate(reader):
            if not row or all((c.strip() == "" for c in row)):
                continue
            if i == 0:
                lower = [c.strip().lower() for c in row]
                if any(k in lower for k in ("score", "relevance", "label", "rel")):
                    header = lower
                    def _idx(names, default):
                        for n in names:
                            if n in header:
                                return header.index(n)
                        return default
                    qid_idx = _idx(("query-id", "qid", "query_id"), 0)
                    did_idx = _idx(("corpus-id", "docid", "doc_id", "did"), 1)
                    rel_idx = _idx(("score", "relevance", "label", "rel"), 2)
                    continue
            try:
                qid = row[qid_idx].strip()
                did = row[did_idx].strip()
                rel_str = row[rel_idx].strip()
            except IndexError:
                continue
            try:
                rel_val = int(float(rel_str))
            except ValueError:
                continue
            rel = 1 if rel_val > 0 else 0
            if rel > 0:
                qrels[qid][did] = rel
    return qrels

def add_prefix(s: str, kind: str, mode: str = "plain") -> str:
    if not USE_PREFIXES:
        return s
    if kind == "query":
        if mode == "neutral_inst":
            return f"query: {s} Используй точное соответствие смысла запроса и отрывка; не делай выводов."
        else:
            return f"query: {s}"
    else:
        return f"passage: {s}"

@torch.no_grad()
def encode_texts(model, tok, texts: List[str], batch_size=BATCH_SIZE, max_len=MAX_LEN, device=DEVICE):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding", leave=False):
        batch = texts[i:i+batch_size]
        enc = tok(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        for k in enc: enc[k] = enc[k].to(device)
        out = model(**enc).last_hidden_state
        mask = enc["attention_mask"].unsqueeze(-1).expand_as(out).float()
        pooled = (out * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)
        pooled = F.normalize(pooled, p=2, dim=1)
        all_embs.append(pooled.detach().cpu())
    return torch.cat(all_embs, dim=0) if all_embs else torch.empty(0, model.config.hidden_size)

def build_index(embs_cpu: torch.Tensor):
    try:
        import faiss
        index = faiss.IndexFlatIP(embs_cpu.shape[1])
        index.add(embs_cpu.numpy().astype("float32"))
        return index, True
    except Exception:
        return embs_cpu, False

@torch.no_grad()
def search(index, is_faiss: bool, q_embs: torch.Tensor, topk: int):
    if is_faiss:
        import faiss
        D, I = index.search(q_embs.numpy().astype("float32"), topk)
        return I, D
    q = q_embs
    d = index
    I_all, D_all = [], []
    chunk = 256
    for i in tqdm(range(0, q.shape[0], chunk), desc="Searching", leave=False):
        sims = torch.mm(q[i:i+chunk], d.t())
        D, I = torch.topk(sims, k=min(topk, sims.size(1)), dim=1)
        I_all.append(I.numpy()); D_all.append(D.numpy())
    return np.vstack(I_all), np.vstack(D_all)

def eval_multi_k(run, qrels: Dict[str, Dict[str,int]], ks: List[int]):
    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks:
        stats[f"Recall@{k}"] = 0.0
        stats[f"MRR@{k}"]    = 0.0
        stats[f"nDCG@{k}"]   = 0.0
        stats[f"MAP@{k}"]    = 0.0   

    n = 0
    for qid, ranking in run.items():
        if qid not in qrels: 
            continue
        n += 1
        rels = qrels[qid]  
        pos_ranks = [r for r,(did,_) in enumerate(ranking, start=1) if did in rels and rels[did] > 0]
        pos_ranks.sort()

        if pos_ranks and pos_ranks[0] == 1:
            stats["Hit@1"] += 1.0

        m_total = sum(1 for _ in rels if rels[_] > 0)  

        for k in ks:
            stats[f"Recall@{k}"] += float(any(r <= k for r in pos_ranks))
            stats[f"MRR@{k}"] += (1.0/pos_ranks[0]) if (pos_ranks and pos_ranks[0] <= k) else 0.0
            dcg = sum(1.0 / math.log2(r+1) for r in pos_ranks if r <= k)
            idcg = sum(1.0 / math.log2(r+1) for r in range(1, min(m_total, k)+1))
            stats[f"nDCG@{k}"] += (dcg / idcg) if idcg > 0 else 0.0

            # MAP@k
            m_k = min(m_total, k)
            if m_k > 0:
                hits = 0
                ap_sum = 0.0
                for r in pos_ranks:
                    if r > k:
                        break
                    hits += 1
                    ap_sum += hits / r
                stats[f"MAP@{k}"] += ap_sum / m_k
            else:
                stats[f"MAP@{k}"] += 0.0

    if n == 0:
        return stats
    stats["N"] = n
    stats["Hit@1"] /= n
    for k in ks:
        stats[f"Recall@{k}"] /= n
        stats[f"MRR@{k}"]    /= n
        stats[f"nDCG@{k}"]   /= n
        stats[f"MAP@{k}"]    /= n
    return stats

@torch.no_grad()
def evaluate(model_dir_or_name: str, tag: str, doc_texts: List[str], q_texts: List[str],
             qids_eval: List[str], qrels: Dict[str, Dict[str,int]], ks: List[int], doc_ids: List[str],
             cache_key: str = None):
    print(f"\n=== {tag} ===")
    tok = AutoTokenizer.from_pretrained(model_dir_or_name, use_fast=True)
    mdl = AutoModel.from_pretrained(model_dir_or_name).to(DEVICE).eval()

    cache_path = None
    if cache_key:
        cache_path = os.path.join(CACHE_DIR, f"{cache_key}__{tag.replace(' ','_')}.pt")
    if cache_path and os.path.exists(cache_path):
        doc_embs = torch.load(cache_path, map_location="cpu")
    else:
        doc_embs = encode_texts(mdl, tok, doc_texts)
        if cache_path: torch.save(doc_embs, cache_path)

    index, is_faiss = build_index(doc_embs)

    q_embs = encode_texts(mdl, tok, q_texts)
    kmax = max(ks)
    I, D = search(index, is_faiss, q_embs, kmax)

    run = {}
    for i, qid in enumerate(qids_eval):
        ids = I[i]; scores = D[i]
        run[qid] = [(doc_ids[j], float(scores[k])) for k, j in enumerate(ids)]
    m = eval_multi_k(run, qrels, ks)
    print({k:(round(v,4) if isinstance(v,float) else v) for k,v in m.items()})
    return m

def run_on_dataset(name: str, spec: dict):
    base_dir = fetch_raw_dataset(name, spec["data_repo"])
    qrels_path = fetch_qrels(name, spec["qrels_repo"], spec["qrels_file"])

    corpus  = load_corpus(os.path.join(base_dir, "corpus.jsonl"))
    queries = load_queries(os.path.join(base_dir, "queries.jsonl"))
    if DOC_CAP:
        rng = random.Random(SEED); doc_ids_all = list(corpus.keys()); rng.shuffle(doc_ids_all)
        keep = set(doc_ids_all[:DOC_CAP])
        corpus = {k:v for k,v in corpus.items() if k in keep}

    qrels = load_qrels_tsv(qrels_path)
    qrels = {qid: {did:rel for did,rel in dd.items() if did in corpus} for qid,dd in qrels.items()}
    qrels = {qid: dd for qid,dd in qrels.items() if dd}

    qids = [qid for qid in qrels.keys() if qid in queries]
    if FAST_Q_CAP and len(qids) > FAST_Q_CAP:
        rng = random.Random(SEED); rng.shuffle(qids); qids = qids[:FAST_Q_CAP]

    print(f"Dataset: {name} | Eval queries: {len(qids)} | Docs: {len(corpus)}")

    doc_ids   = list(corpus.keys())
    doc_texts = [add_prefix(corpus[d], "passage") for d in doc_ids]
    q_plain   = [add_prefix(queries[q], "query", mode="plain") for q in qids]
    q_neutral = [add_prefix(queries[q], "query", mode="neutral_inst") for q in qids]

    results = {}

    if FINETUNED_INST_DIR and os.path.exists(FINETUNED_INST_DIR):
        results["finetuned_instruction"] = evaluate(
            FINETUNED_INST_DIR, "Finetuned (instruction-aware)",
            doc_texts, q_neutral, qids, qrels, K_LIST, doc_ids,
            cache_key=f"{name}"
        )

    if FINETUNED_QUERY_DIR and os.path.exists(FINETUNED_QUERY_DIR):
        results["finetuned_queryonly"] = evaluate(
            FINETUNED_QUERY_DIR, "Finetuned (query-only)",
            doc_texts, q_plain, qids, qrels, K_LIST, doc_ids,
            cache_key=f"{name}"
        )

    results["baseline"] = evaluate(
        BASE_MODEL, "Baseline (zero-shot)",
        doc_texts, q_plain, qids, qrels, K_LIST, doc_ids,
        cache_key=f"{name}"
    )

    print("\n--- Summary:", name, "---")
    for k, v in results.items():
        print(k, "->", {kk:(round(vv,3) if isinstance(vv,float) else vv) for kk,vv in (v or {}).items()})
    return results

all_results = {}
for name, spec in SETS.items():
    print(f"\n=== Fetch & Evaluate: {name} ===")
    all_results[name] = run_on_dataset(name, spec)

print("\n=== GRAND SUMMARY ===")
for ds, res in all_results.items():
    print("\n", ds)
    for k, v in res.items():
        print(" ", k, "->", {kk:(round(vv,3) if isinstance(vv,float) else vv) for kk,vv in (v or {}).items()})


Device: cuda

=== Fetch & Evaluate: rus-xquad ===


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


corpus.jsonl: 0.00B [00:00, ?B/s]

queries.jsonl: 0.00B [00:00, ?B/s]

...

dev.tsv: 0.00B [00:00, ?B/s]

Dataset: wikifacts-articles | Eval queries: 300 | Docs: 12848

=== Finetuned (instruction-aware) ===


{'N': 300, 'Hit@1': 0.49, 'Recall@2': 0.6567, 'MRR@2': 0.5733, 'nDCG@2': 0.511, 'MAP@2': 0.4708, 'Recall@5': 0.74, 'MRR@5': 0.5975, 'nDCG@5': 0.5559, 'MAP@5': 0.4987, 'Recall@10': 0.8033, 'MRR@10': 0.6062, 'nDCG@10': 0.585, 'MAP@10': 0.5143}

=== Finetuned (query-only) ===


{'N': 300, 'Hit@1': 0.42, 'Recall@2': 0.5633, 'MRR@2': 0.4917, 'nDCG@2': 0.4389, 'MAP@2': 0.4042, 'Recall@5': 0.69, 'MRR@5': 0.5274, 'nDCG@5': 0.4914, 'MAP@5': 0.4322, 'Recall@10': 0.73, 'MRR@10': 0.5327, 'nDCG@10': 0.5146, 'MAP@10': 0.4452}

=== Baseline (zero-shot) ===


{'N': 300, 'Hit@1': 0.4467, 'Recall@2': 0.6067, 'MRR@2': 0.5267, 'nDCG@2': 0.4712, 'MAP@2': 0.4342, 'Recall@5': 0.6967, 'MRR@5': 0.5519, 'nDCG@5': 0.5172, 'MAP@5': 0.4624, 'Recall@10': 0.78, 'MRR@10': 0.5629, 'nDCG@10': 0.5501, 'MAP@10': 0.4787}

--- Summary: wikifacts-articles ---
finetuned_instruction -> {'N': 300, 'Hit@1': 0.49, 'Recall@2': 0.657, 'MRR@2': 0.573, 'nDCG@2': 0.511, 'MAP@2': 0.471, 'Recall@5': 0.74, 'MRR@5': 0.598, 'nDCG@5': 0.556, 'MAP@5': 0.499, 'Recall@10': 0.803, 'MRR@10': 0.606, 'nDCG@10': 0.585, 'MAP@10': 0.514}
finetuned_queryonly -> {'N': 300, 'Hit@1': 0.42, 'Recall@2': 0.563, 'MRR@2': 0.492, 'nDCG@2': 0.439, 'MAP@2': 0.404, 'Recall@5': 0.69, 'MRR@5': 0.527, 'nDCG@5': 0.491, 'MAP@5': 0.432, 'Recall@10': 0.73, 'MRR@10': 0.533, 'nDCG@10': 0.515, 'MAP@10': 0.445}
baseline -> {'N': 300, 'Hit@1': 0.447, 'Recall@2': 0.607, 'MRR@2': 0.527, 'nDCG@2': 0.471, 'MAP@2': 0.434, 'Recall@5': 0.697, 'MRR@5': 0.552, 'nDCG@5': 0.517, 'MAP@5': 0.462, 'Recall@10': 0.78, 'MRR@10': 

In [27]:
# Evaluate: rus-arguana & rus-scifact (RU) with LoRA adapters
# - Baseline E5-base, LoRA instruction-aware, LoRA query-only
# - Rebuild qrels from BEIR via ir_datasets, filtered to RU IDs
# - Caches doc embeddings per (dataset, model)

import os, io, gzip, glob, json, math, random, sys, hashlib
from collections import defaultdict
from typing import Dict, List, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import snapshot_download
from tqdm import tqdm

import ir_datasets as irds

try:
    import faiss
    FAISS_OK = True
except Exception:
    FAISS_OK = False

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE    = 64           
MAX_LEN       = 256
K_LIST        = [1, 5, 10]
SEED          = 42
FAST_Q_CAP    = 300           
DOC_CAP       = None         

FINETUNED_INST_DIR  = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"    
FINETUNED_QUERY_DIR = "/kaggle/working/mE5_lora8bit_queryonly_v1/best"   
BASE_MODEL          = "intfloat/multilingual-e5-base"                    

USE_PREFIXES       = True
USE_NEUTRAL_INST   = True  
NEUTRAL_RU         = "Используй точное соответствие смысла запроса и отрывка; не делай выводов."

CACHE_DIR   = "/kaggle/working/rusbeir_eval_cache"
OUT_ROOT    = "/kaggle/working/rusbeir_eval_runs"
HF_CACHE    = "/kaggle/working/hf_cache"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUT_ROOT,  exist_ok=True)
os.makedirs(HF_CACHE,  exist_ok=True)

print(f"Device: {DEVICE} | BATCH={BATCH_SIZE} | MAX_LEN={MAX_LEN} | FAST_Q_CAP={FAST_Q_CAP} | DOC_CAP={DOC_CAP}")

SETS = {
    "rus-arguana": {
        "data_repo": "kaengreg/rus-arguana",
        "beir_ids":  ["beir/arguana", "beir-v1.0.0/arguana", "arguana"],   # try in order
    },
    "rus-scifact": {
        "data_repo": "kaengreg/rus-scifact",
        "beir_ids":  ["beir/scifact/test", "beir/scifact"],                # try in order
    },
}

def _copy(src, dst):
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    if src == dst:
        return
    with open(src, "rb") as f_in, open(dst, "wb") as f_out:
        f_out.write(f_in.read())

def _maybe_gunzip(src, dst):
    if src.endswith(".gz"):
        with gzip.open(src, "rt", encoding="utf-8") as f_in, open(dst, "w", encoding="utf-8") as f_out:
            for line in f_in: f_out.write(line)
    else:
        _copy(src, dst)

def fetch_raw_dataset(name: str, data_repo: str) -> str:
    out_dir = os.path.join(OUT_ROOT, name)
    os.makedirs(out_dir, exist_ok=True)
    root = snapshot_download(repo_id=data_repo, repo_type="dataset",
                             allow_patterns=["corpus.jsonl", "queries.jsonl", "*.gz"],
                             local_dir=HF_CACHE, local_dir_use_symlinks=False)
    cand_corpus  = glob.glob(os.path.join(root, "**", "corpus.jsonl*"),  recursive=True)
    cand_queries = glob.glob(os.path.join(root, "**", "queries.jsonl*"), recursive=True)
    if not cand_corpus or not cand_queries:
        raise FileNotFoundError(f"Could not find corpus.jsonl/queries.jsonl in {data_repo}")
    _maybe_gunzip(sorted(cand_corpus)[-1],  os.path.join(out_dir, "corpus.jsonl"))
    _maybe_gunzip(sorted(cand_queries)[-1], os.path.join(out_dir, "queries.jsonl"))
    return out_dir

def smart_open(path):
    if path.endswith(".gz"):
        return io.TextIOWrapper(gzip.open(path, "rb"), encoding="utf-8")
    return open(path, "r", encoding="utf-8")

def load_corpus(path: str) -> Dict[str, str]:
    corpus = {}
    with smart_open(path) as f:
        for line in f:
            j = json.loads(line)
            cid   = str(j.get("_id") or j.get("id"))
            title = (j.get("title") or "").strip()
            text  = (j.get("text")  or "").strip()
            corpus[cid] = (title + " " + text).strip()
    return corpus

def load_queries(path: str) -> Dict[str, str]:
    queries = {}
    with smart_open(path) as f:
        for line in f:
            j = json.loads(line)
            qid  = str(j.get("_id") or j.get("id"))
            text = (j.get("text") or "").strip()
            queries[qid] = text
    return queries

def build_qrels_from_beir(beir_ids: List[str], hf_qids: set, hf_docs: set):
    ds = None; chosen = None
    tried = []
    for dsid in beir_ids:
        tried.append(dsid)
        try:
            ds = irds.load(dsid)
            chosen = dsid
            break
        except KeyError:
            continue
    if ds is None:
        raise KeyError(f"None of these IRDS ids exist: {tried}")

    qrels = defaultdict(dict)
    kept = skipped = 0
    for q in ds.qrels_iter():
        qid = str(q.query_id)
        did = str(q.doc_id)
        rel = int(getattr(q, "relevance", 1))
        if (qid in hf_qids) and (did in hf_docs) and rel > 0:
            qrels[qid][did] = 1
            kept += 1
        else:
            skipped += 1
    print(f"  qrels from {chosen} -> kept {kept:,}, skipped {skipped:,}")
    return dict(qrels)

def add_prefix_query(s: str, for_inst_model: bool) -> str:
    if not USE_PREFIXES:
        return s
    q = f"query: {s}".strip()
    if for_inst_model and USE_NEUTRAL_INST and NEUTRAL_RU:
        q = f"{q} {NEUTRAL_RU}".strip()
    return q

def add_prefix_passage(s: str) -> str:
    if not USE_PREFIXES:
        return s
    return f"passage: {s}".strip()

def corpus_signature(ids: List[str]) -> str:
    key = "||".join(ids[:50]) + "||" + "||".join(ids[-50:]) + f"||{len(ids)}"
    return hashlib.md5(key.encode()).hexdigest()[:8]

@torch.no_grad()
def encode_texts(model, tok, texts: List[str], batch_size=BATCH_SIZE, max_len=MAX_LEN, device=DEVICE):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc=f"Encoding@{device}", leave=False):
        batch = texts[i:i+batch_size]
        enc = tok(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        for k in enc: enc[k] = enc[k].to(device)
        out = model(**enc).last_hidden_state   # [B, L, H]
        mask = enc["attention_mask"].unsqueeze(-1).expand_as(out).float()
        pooled = (out * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)  # mean pool
        pooled = F.normalize(pooled, p=2, dim=1)
        all_embs.append(pooled.detach().cpu())
    return torch.cat(all_embs, dim=0) if all_embs else torch.empty(0, model.config.hidden_size)

def build_index(embs_cpu: torch.Tensor):
    if FAISS_OK:
        index = faiss.IndexFlatIP(embs_cpu.shape[1])
        index.add(embs_cpu.numpy().astype("float32"))
        return index, True
    return embs_cpu, False

@torch.no_grad()
def search(index, is_faiss: bool, q_embs: torch.Tensor, topk: int):
    if is_faiss:
        D, I = index.search(q_embs.numpy().astype("float32"), topk)
        return I, D
    # torch fallback
    q = q_embs; d = index
    I_all, D_all = [], []
    chunk = 256
    for i in tqdm(range(0, q.shape[0], chunk), desc="Searching", leave=False):
        sims = torch.mm(q[i:i+chunk], d.t())
        D, I = torch.topk(sims, k=min(topk, sims.size(1)), dim=1)
        I_all.append(I.numpy()); D_all.append(D.numpy())
    return np.vstack(I_all), np.vstack(D_all)

def eval_multi_k(run, qrels: Dict[str, Dict[str,int]], ks: List[int]):
    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks:
        stats[f"Recall@{k}"] = 0.0
        stats[f"MRR@{k}"]    = 0.0
        stats[f"nDCG@{k}"]   = 0.0
        stats[f"MAP@{k}"]    = 0.0   

    n = 0
    for qid, ranking in run.items():
        if qid not in qrels: 
            continue
        n += 1
        rels = qrels[qid]
        pos_ranks = [r for r,(did,_) in enumerate(ranking, start=1) if did in rels and rels[did] > 0]
        pos_ranks.sort()

        if pos_ranks and pos_ranks[0] == 1:
            stats["Hit@1"] += 1.0

        m_total = sum(1 for _ in rels if rels[_] > 0)  

        for k in ks:
            stats[f"Recall@{k}"] += float(any(r <= k for r in pos_ranks))
            stats[f"MRR@{k}"]    += (1.0/pos_ranks[0]) if (pos_ranks and pos_ranks[0] <= k) else 0.0
            dcg  = sum(1.0/math.log2(r+1) for r in pos_ranks if r <= k)
            idcg = sum(1.0/math.log2(r+1) for r in range(1, min(m_total, k)+1))
            stats[f"nDCG@{k}"] += (dcg/idcg) if idcg > 0 else 0.0
            m_k = min(m_total, k)
            if m_k > 0:
                hits = 0
                ap_sum = 0.0
                for r in pos_ranks:
                    if r > k:
                        break
                    hits += 1
                    ap_sum += hits / r
                stats[f"MAP@{k}"] += ap_sum / m_k
            else:
                stats[f"MAP@{k}"] += 0.0

    if n == 0:
        return stats
    stats["N"] = n
    stats["Hit@1"] /= n
    for k in ks:
        stats[f"Recall@{k}"] /= n
        stats[f"MRR@{k}"]    /= n
        stats[f"nDCG@{k}"]   /= n
        stats[f"MAP@{k}"]    /= n
    return stats

def fmt(d):  
    return {k:(round(v,4) if isinstance(v,float) else v) for k,v in d.items()}

def safe_load_peft_for_inference(base_model_name: str, adapter_dir: str, device: str):

    from peft import PeftModel
    torch.cuda.empty_cache()
    dtype = torch.float16 if (device=="cuda") else None
    try:
        base = AutoModel.from_pretrained(base_model_name, torch_dtype=dtype, low_cpu_mem_usage=True)
        base = base.to(device).eval()
    except RuntimeError as e:
        print(f"[warn] base on GPU failed ({type(e).__name__}): using CPU fp32")
        base = AutoModel.from_pretrained(base_model_name, low_cpu_mem_usage=True).to("cpu").eval()
        device = "cpu"
    try:
        tok = AutoTokenizer.from_pretrained(adapter_dir, use_fast=True)
    except Exception:
        tok = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
    model = PeftModel.from_pretrained(base, adapter_dir).eval()
    return tok, model, device

def safe_load_baseline(model_name: str, device: str):
    dtype = torch.float16 if (device=="cuda") else None
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModel.from_pretrained(model_name, torch_dtype=dtype, low_cpu_mem_usage=True).to(device).eval()
        return tok, mdl, device
    except RuntimeError as e:
        print(f"[warn] baseline on GPU failed ({type(e).__name__}): using CPU fp32")
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModel.from_pretrained(model_name, low_cpu_mem_usage=True).to("cpu").eval()
        return tok, mdl, "cpu"

def evaluate_model(tag: str, tok, mdl, dev: str,
                   doc_ids: List[str], doc_texts: List[str],
                   qids_eval: List[str], q_texts: List[str],
                   qrels: Dict[str, Dict[str,int]],
                   cache_key: str):
    print(f"\n=== {tag} ===")

    cache_path = os.path.join(CACHE_DIR, f"docs__{cache_key}.pt")
    if os.path.exists(cache_path):
        doc_embs = torch.load(cache_path, map_location="cpu")
        print(f"[cache] loaded docs {doc_embs.shape} from {cache_path}")
    else:
        doc_embs = encode_texts(mdl, tok, doc_texts, batch_size=BATCH_SIZE, max_len=MAX_LEN, device=dev)
        torch.save(doc_embs, cache_path)
        print(f"[cache] saved docs {doc_embs.shape} -> {cache_path}")

    index, is_faiss = build_index(doc_embs)

    q_embs = encode_texts(mdl, tok, q_texts, batch_size=BATCH_SIZE, max_len=MAX_LEN, device=dev)

    I, D = search(index, is_faiss, q_embs, topk=max(K_LIST))

    run = {}
    for i, qid in enumerate(qids_eval):
        ids = I[i]; scores = D[i]
        run[qid] = [(doc_ids[j], float(scores[k])) for k, j in enumerate(ids)]

    m = eval_multi_k(run, qrels, K_LIST)
    print(fmt(m))
    return m

def run_on_dataset(name: str, spec: dict):
    base_dir = fetch_raw_dataset(name, spec["data_repo"])
    corpus   = load_corpus(os.path.join(base_dir, "corpus.jsonl"))
    queries  = load_queries(os.path.join(base_dir, "queries.jsonl"))

    if DOC_CAP:
        rng = random.Random(SEED)
        keys = list(corpus.keys()); rng.shuffle(keys)
        keep = set(keys[:DOC_CAP])
        corpus = {k:v for k,v in corpus.items() if k in keep}

    hf_qids = set(queries.keys())
    hf_docs = set(corpus.keys())
    qrels   = build_qrels_from_beir(spec["beir_ids"], hf_qids, hf_docs)

    qids = [qid for qid in queries if qid in qrels]
    if FAST_Q_CAP and len(qids) > FAST_Q_CAP:
        rng = random.Random(SEED)
        rng.shuffle(qids)
        qids = qids[:FAST_Q_CAP]

    print(f"\nDataset: {name} | Eval queries: {len(qids)} | Docs: {len(corpus)}")

    doc_ids   = list(corpus.keys())
    doc_texts = [add_prefix_passage(corpus[d]) for d in doc_ids]

    q_texts_base   = [add_prefix_query(queries[q], for_inst_model=False) for q in qids]
    q_texts_inst   = [add_prefix_query(queries[q], for_inst_model=True ) for q in qids]
    q_texts_qonly  = [add_prefix_query(queries[q], for_inst_model=False) for q in qids]  

    results = {}

    tok_b, mdl_b, dev_b = safe_load_baseline(BASE_MODEL, DEVICE)
    cache_key_b = f"{name}__baseline__{corpus_signature(doc_ids)}__L{MAX_LEN}"
    results["baseline"] = evaluate_model("Baseline (mE5-base)", tok_b, mdl_b, dev_b,
                                         doc_ids, doc_texts, qids, q_texts_base, qrels, cache_key_b)
    del mdl_b; torch.cuda.empty_cache()

    if FINETUNED_INST_DIR and os.path.exists(FINETUNED_INST_DIR):
        tok_i, mdl_i, dev_i = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_INST_DIR, DEVICE)
        cache_key_i = f"{name}__inst__{corpus_signature(doc_ids)}__L{MAX_LEN}"
        results["finetuned_instruction"] = evaluate_model("LoRA (instruction-aware)", tok_i, mdl_i, dev_i,
                                                          doc_ids, doc_texts, qids, q_texts_inst, qrels, cache_key_i)
        del mdl_i; torch.cuda.empty_cache()
    else:
        print("(!) Skipping finetuned_instruction — dir not found.")

    if FINETUNED_QUERY_DIR and os.path.exists(FINETUNED_QUERY_DIR):
        tok_q, mdl_q, dev_q = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_QUERY_DIR, DEVICE)
        cache_key_q = f"{name}__queryonly__{corpus_signature(doc_ids)}__L{MAX_LEN}"
        results["finetuned_queryonly"] = evaluate_model("LoRA (query-only)", tok_q, mdl_q, dev_q,
                                                        doc_ids, doc_texts, qids, q_texts_qonly, qrels, cache_key_q)
        del mdl_q; torch.cuda.empty_cache()
    else:
        print("(!) Skipping finetuned_queryonly — dir not found.")

    print("\n--- Summary:", name, "---")
    for k, v in results.items():
        print(k, "->", fmt(v))
    return results

all_results = {}
for name, spec in SETS.items():
    all_results[name] = run_on_dataset(name, spec)

print("\n=== GRAND SUMMARY ===")
for ds, res in all_results.items():
    print("\n", ds)
    for k, v in res.items():
        print(" ", k, "->", fmt(v))


Device: cuda | BATCH=64 | MAX_LEN=256 | FAST_Q_CAP=300 | DOC_CAP=None


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

queries.jsonl: 0.00B [00:00, ?B/s]

corpus.jsonl:   0%|          | 0.00/33.8M [00:00<?, ?B/s]

[INFO] [starting] opening zip file
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/arguana.zip
[INFO] [finished] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/arguana.zip: [00:00] [3.77MB] [4.83MB/s]
[INFO] [finished] opening zip file [5.25s]                                                                 


...

{'N': 300, 'Hit@1': 0.49, 'Recall@1': 0.49, 'MRR@1': 0.49, 'nDCG@1': 0.49, 'MAP@1': 0.49, 'Recall@5': 0.69, 'MRR@5': 0.5646, 'nDCG@5': 0.5845, 'MAP@5': 0.5496, 'Recall@10': 0.75, 'MRR@10': 0.5729, 'nDCG@10': 0.6073, 'MAP@10': 0.5608}

=== LoRA (instruction-aware) ===


[cache] saved docs torch.Size([5183, 768]) -> /kaggle/working/rusbeir_eval_cache/docs__rus-scifact__inst__fe705ae6__L256.pt


{'N': 300, 'Hit@1': 0.45, 'Recall@1': 0.45, 'MRR@1': 0.45, 'nDCG@1': 0.45, 'MAP@1': 0.45, 'Recall@5': 0.6667, 'MRR@5': 0.5336, 'nDCG@5': 0.5567, 'MAP@5': 0.5198, 'Recall@10': 0.7267, 'MRR@10': 0.542, 'nDCG@10': 0.58, 'MAP@10': 0.5315}

=== LoRA (query-only) ===


[cache] saved docs torch.Size([5183, 768]) -> /kaggle/working/rusbeir_eval_cache/docs__rus-scifact__queryonly__fe705ae6__L256.pt


{'N': 300, 'Hit@1': 0.4133, 'Recall@1': 0.4133, 'MRR@1': 0.4133, 'nDCG@1': 0.4133, 'MAP@1': 0.4133, 'Recall@5': 0.63, 'MRR@5': 0.494, 'nDCG@5': 0.5155, 'MAP@5': 0.4782, 'Recall@10': 0.6933, 'MRR@10': 0.5029, 'nDCG@10': 0.54, 'MAP@10': 0.4905}

--- Summary: rus-scifact ---
baseline -> {'N': 300, 'Hit@1': 0.49, 'Recall@1': 0.49, 'MRR@1': 0.49, 'nDCG@1': 0.49, 'MAP@1': 0.49, 'Recall@5': 0.69, 'MRR@5': 0.5646, 'nDCG@5': 0.5845, 'MAP@5': 0.5496, 'Recall@10': 0.75, 'MRR@10': 0.5729, 'nDCG@10': 0.6073, 'MAP@10': 0.5608}
finetuned_instruction -> {'N': 300, 'Hit@1': 0.45, 'Recall@1': 0.45, 'MRR@1': 0.45, 'nDCG@1': 0.45, 'MAP@1': 0.45, 'Recall@5': 0.6667, 'MRR@5': 0.5336, 'nDCG@5': 0.5567, 'MAP@5': 0.5198, 'Recall@10': 0.7267, 'MRR@10': 0.542, 'nDCG@10': 0.58, 'MAP@10': 0.5315}
finetuned_queryonly -> {'N': 300, 'Hit@1': 0.4133, 'Recall@1': 0.4133, 'MRR@1': 0.4133, 'nDCG@1': 0.4133, 'MAP@1': 0.4133, 'Recall@5': 0.63, 'MRR@5': 0.494, 'nDCG@5': 0.5155, 'MAP@5': 0.4782, 'Recall@10': 0.6933, 'MRR@10'

In [28]:
# Unified Robustness / Prompt-Transfer Eval 
#  - Datasets: Mr.TyDi-RU (prepared), mFollowIR-RU, LAReQA XQuAD-R (RU↔EN)
#  - Models:   mE5-base, optional LoRA inst-aware, LoRA query-only
#  - Metrics:  Hit@1, Recall@K, MRR@K, MAP@K, nDCG@K, p-MRR@K
#  - Robustness: Prompt ablation, Paraphrase variance (5x), Light noise stress

import os, io, re, json, math, time, random, hashlib, gzip, glob
from collections import defaultdict
from typing import Dict, List, Tuple

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

try:
    import faiss
    FAISS_OK = True
except Exception:
    FAISS_OK = False

DEVICE        = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE_D  = 64
BATCH_SIZE_Q  = 64
MAX_LEN       = 256
K_LIST        = [1, 5, 10]
SEED          = 42

DO_PROMPT_ABLATION   = True    
DO_PARAPHRASE_ROBUST = True    
DO_NOISE_STRESS      = True    

BASE_MODEL          = "intfloat/multilingual-e5-base"
FINETUNED_INST_DIR  = "/kaggle/working/mE5_lora8bit_infoNCE_v1/best"     
FINETUNED_QUERY_DIR = "/kaggle/working/mE5_lora8bit_queryonly_v1/best"   

USE_PREFIXES = True  

ROOT_WORK         = "/kaggle/working"
CACHE_DIR_EMB     = os.path.join(ROOT_WORK, "robust_eval_cache"); os.makedirs(CACHE_DIR_EMB, exist_ok=True)

MRTYDI_DIR        = os.path.join(ROOT_WORK, "mrtydi_ru_prepared")
LAREQA_DIR        = os.path.join(ROOT_WORK, "lareqa_xquad_r"); os.makedirs(LAREQA_DIR, exist_ok=True)

MRTYDI_DOC_CAP    = 20000
MRTYDI_FAST_Q     = None
MFOLLOWIR_DOC_CAP = None
MFOLLOWIR_FAST_Q  = None
LAREQA_DOC_CAP    = None
LAREQA_FAST_Q     = None

random.seed(SEED); np.random.seed(SEED)
print(f"Device: {DEVICE} | K={K_LIST} | MAX_LEN={MAX_LEN}")

PROMPTS_RU = {
    "plain": "",
    "minimal_ru": "Верни релевантные документы.",
    "qa_ru":      "Найди предложение, отвечающее на вопрос.",
    "search_ru":  "Найди веб-страницы, связанные с запросом.",
    "mfoll_ru":   "Ответь кратко и по делу. Найди наиболее релевантные документы."
}
PROMPTS_EN = {
    "plain": "",
    "minimal_en": "Return relevant documents.",
    "qa_en":      "Find the sentence that answers the question.",
    "search_en":  "Find web pages related to the query.",
    "mfoll_en":   "Answer briefly and to the point. Retrieve the most relevant documents."
}

PARAPHRASES_RU = [
    "Дай наиболее релевантные документы.",
    "Найди лучшие соответствующие тексты.",
    "Верни документы, максимально подходящие запросу.",
    "Подбери самые уместные тексты по запросу.",
    "Найди тексты, которые лучше всего отвечают запросу.",
]
PARAPHRASES_EN = [
    "Return the most relevant documents.",
    "Find the best matching texts.",
    "Provide documents that best fit the query.",
    "Select the most suitable texts for the query.",
    "Retrieve texts that best answer the query.",
]

# Instruction BANK (still used for p-MRR and transfer)
BANK = [
  "Ты — поисковая система. Найди наиболее релевантные документы, помогающие ответить на запрос. Возвращай документы, а не готовый ответ.",
  "Найди пассажи, содержащие факты для ответа на вопрос. Отдавай приоритет точности и контекстной релевантности.",
  "Выполни поиск по корпусу и верни документы, где максимально вероятно встретится ответ. Избегай нерелевантных совпадений."
]
def bank_for_qid(qid: str) -> str:
    return BANK[int(hashlib.md5(qid.encode()).hexdigest(), 16) % len(BANK)]

def add_prefix(s: str, kind: str) -> str:
    if not USE_PREFIXES: return s
    return (("query: " if kind == "query" else "passage: ") + s).strip()

def fmt(d): return {k:(round(v,4) if isinstance(v,float) else v) for k,v in d.items()}

def corpus_signature(ids: List[str]) -> str:
    key = "||".join(ids[:50]) + "||" + "||".join(ids[-50:]) + f"||{len(ids)}"
    return hashlib.md5(key.encode()).hexdigest()[:8]

def noise_variants(q: str) -> Dict[str, str]:
    base = re.sub(r"[^\w\s]", " ", q)
    base = re.sub(r"\s+", " ", base).strip()
    toks = base.split()
    swapped = base
    if len(toks) >= 2:
        t = toks[:]
        i = random.randrange(0, len(t)-1)
        t[i], t[i+1] = t[i+1], t[i]
        swapped = " ".join(t)
    return {"clean": q, "nopunct": base, "swap": swapped, "lower": q.lower()}

def mean_pool(last_hidden_state, attention_mask):
    m = attention_mask.unsqueeze(-1).expand_as(last_hidden_state).float()
    return (last_hidden_state * m).sum(dim=1) / m.sum(dim=1).clamp(min=1e-9)

@torch.no_grad()
def encode_texts(model, tok, texts: List[str], batch_size: int, max_len: int, device: str):
    outs=[]
    for i in tqdm(range(0, len(texts), batch_size), desc=f"Encode@{device}", leave=False):
        batch = texts[i:i+batch_size]
        enc = tok(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        for k in enc: enc[k] = enc[k].to(device)
        last = model(**enc).last_hidden_state
        pooled = mean_pool(last, enc["attention_mask"])
        outs.append(F.normalize(pooled, p=2, dim=1).cpu())
    return torch.cat(outs, 0) if outs else torch.empty(0, model.config.hidden_size)

def build_index(doc_embs_cpu: torch.Tensor):
    if FAISS_OK:
        index = faiss.IndexFlatIP(doc_embs_cpu.shape[1])
        index.add(doc_embs_cpu.numpy().astype("float32"))
        return index, True
    return doc_embs_cpu, False

@torch.no_grad()
def search_index(index, is_faiss: bool, q_embs: torch.Tensor, topk: int):
    if is_faiss:
        D, I = index.search(q_embs.numpy().astype("float32"), topk)
        return I, D
    sims = torch.mm(q_embs, index.t())
    D, I = torch.topk(sims, k=min(topk, sims.size(1)), dim=1)
    return I.numpy(), D.numpy()

def eval_multi_k_and_perquery(run, qrels: Dict[str, Dict[str,int]], ks: List[int]):
    ks = sorted(set(ks))
    stats = {"N": 0, "Hit@1": 0.0}
    for k in ks:
        stats.update({f"Recall@{k}":0.0, f"MRR@{k}":0.0, f"MAP@{k}":0.0, f"nDCG@{k}":0.0})
    per_mrr = {k:{} for k in ks}
    per_ap  = {k:{} for k in ks}
    n=0
    for qid, ranking in run.items():
        if qid not in qrels: continue
        n += 1
        rels = qrels[qid]
        pos = [r for r,(did,_) in enumerate(ranking, start=1) if did in rels]
        pos.sort()
        if pos and pos[0]==1: stats["Hit@1"] += 1.0
        m_rel = len(rels)
        for k in ks:
            stats[f"Recall@{k}"] += float(any(r<=k for r in pos))
            mrr_q = (1.0/pos[0]) if (pos and pos[0]<=k) else 0.0
            stats[f"MRR@{k}"] += mrr_q
            per_mrr[k][qid] = mrr_q
            dcg  = sum(1.0/math.log2(r+1) for r in pos if r<=k)
            idcg = sum(1.0/math.log2(r+1) for r in range(1, min(m_rel,k)+1))
            stats[f"nDCG@{k}"] += (dcg/idcg) if idcg>0 else 0.0
            hits=0; ap_sum=0.0
            for r,(did,_) in enumerate(ranking[:k], start=1):
                if did in rels:
                    hits += 1; ap_sum += hits / r
            denom = float(min(m_rel, k)) if m_rel>0 else 1.0
            ap_q = (ap_sum/denom) if denom>0 else 0.0
            stats[f"MAP@{k}"] += ap_q
            per_ap[k][qid] = ap_q
    if n==0: return stats, per_mrr, per_ap
    stats["N"]=n; stats["Hit@1"] /= n
    for k in ks:
        stats[f"Recall@{k}"] /= n; stats[f"MRR@{k}"]/=n; stats[f"MAP@{k}"]/=n; stats[f"nDCG@{k}"]/=n
    return stats, per_mrr, per_ap

def paired_diff_mean(per_a: Dict[str,float], per_b: Dict[str,float]) -> float:
    qids = sorted(set(per_a.keys()) & set(per_b.keys()))
    if not qids: return 0.0
    return float(np.mean([per_b[q]-per_a[q] for q in qids]))

from transformers import AutoTokenizer, AutoModel
def safe_load_baseline(model_name: str, device: str):
    dtype = torch.float16 if (device=="cuda") else None
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModel.from_pretrained(model_name, torch_dtype=dtype, low_cpu_mem_usage=True).to(device).eval()
    except RuntimeError:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModel.from_pretrained(model_name, low_cpu_mem_usage=True).to("cpu").eval()
        device = "cpu"
    return tok, mdl, device

def safe_load_peft_for_inference(base_model_name: str, adapter_dir: str, device: str):
    try:
        import peft.import_utils as _iu
        _iu.is_bnb_available = lambda: False
    except Exception:
        pass
    from peft import PeftModel
    dtype = torch.float16 if (device=="cuda") else None
    try:
        base = AutoModel.from_pretrained(base_model_name, torch_dtype=dtype, low_cpu_mem_usage=True).to(device).eval()
    except RuntimeError:
        base = AutoModel.from_pretrained(base_model_name, low_cpu_mem_usage=True).to("cpu").eval()
        device = "cpu"
    try:
        tok = AutoTokenizer.from_pretrained(adapter_dir, use_fast=True)
    except Exception:
        tok = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
    model = PeftModel.from_pretrained(base, adapter_dir).eval()
    return tok, model, device

def evaluate_model(tag: str, tok, mdl, dev: str,
                   doc_ids: List[str], doc_texts_pref: List[str],
                   qids: List[str], q_texts_pref: List[str],
                   qrels: Dict[str, Dict[str,int]],
                   cache_key: str):
    print(f"\n=== {tag} ===")
    dcache = os.path.join(CACHE_DIR_EMB, f"docs__{cache_key}.pt")
    if os.path.exists(dcache):
        doc_embs = torch.load(dcache, map_location="cpu"); print(f"[cache] docs {doc_embs.shape} loaded")
    else:
        doc_embs = encode_texts(mdl, tok, doc_texts_pref, BATCH_SIZE_D, MAX_LEN, dev)
        torch.save(doc_embs, dcache); print(f"[cache] docs {doc_embs.shape} saved -> {dcache}")
    index, is_faiss = build_index(doc_embs)
    q_embs = encode_texts(mdl, tok, q_texts_pref, BATCH_SIZE_Q, MAX_LEN, dev)
    I, D = search_index(index, is_faiss, q_embs, topk=max(K_LIST))
    run = { qid: [(doc_ids[j], float(D[i][k])) for k, j in enumerate(I[i])] for i, qid in enumerate(qids) }
    stats, per_mrr, per_ap = eval_multi_k_and_perquery(run, qrels, K_LIST)
    print(fmt(stats))
    return stats, per_mrr, per_ap

def build_prompted_queries(qids: List[str], qmap: Dict[str,str], prompt_text: str):
    if prompt_text.strip()=="":
        return [add_prefix(qmap[q], "query") for q in qids]
    return [add_prefix((qmap[q] + " " + prompt_text).strip(), "query") for q in qids]

def paraphrase_suite(qids: List[str], qmap: Dict[str,str], paraphrases: List[str]):
    return {f"para{i+1}": [add_prefix((qmap[q] + " " + p).strip(), "query") for q in qids]
            for i,p in enumerate(paraphrases)}

def noise_suite(qids: List[str], qmap: Dict[str,str]):
    suite = {"clean": [add_prefix(qmap[q], "query") for q in qids]}
    # compute per-q variants and then stitch column-wise
    per = {q: noise_variants(qmap[q]) for q in qids}
    for key in ["nopunct","swap","lower"]:
        suite[key] = [add_prefix(per[q][key], "query") for q in qids]
    return suite

def agg_mean_std(stats_list: List[Dict[str,float]]):
    if not stats_list: return {}
    keys = stats_list[0].keys()
    out = {}
    for k in keys:
        if isinstance(stats_list[0][k], float):
            vals = [s[k] for s in stats_list]
            out[f"{k}_mean"] = float(np.mean(vals))
            out[f"{k}_std"]  = float(np.std(vals))
    return out

# A) Mr.TyDi-RU (prepared)

def prepare_mrtydi_ru(out_dir: str):
    if all(os.path.exists(os.path.join(out_dir, fn)) for fn in ["corpus.jsonl","queries.jsonl","qrels.tsv"]):
        print("[MrTyDi] prepared files already present."); return
    print("[MrTyDi] preparing via ir_datasets …")
    import ir_datasets as irds
    os.makedirs(out_dir, exist_ok=True)
    ds = irds.load("mr-tydi/ru/test")
    with open(os.path.join(out_dir, "corpus.jsonl"), "w", encoding="utf-8") as fc:
        for d in ds.docs_iter():
            j = {"_id": str(d.doc_id), "title": getattr(d, "title", "") or "", "text": d.text or ""}
            fc.write(json.dumps(j, ensure_ascii=False) + "\n")
    with open(os.path.join(out_dir, "queries.jsonl"), "w", encoding="utf-8") as fq:
        for q in ds.queries_iter():
            j = {"_id": str(q.query_id), "text": q.text}
            fq.write(json.dumps(j, ensure_ascii=False) + "\n")
    with open(os.path.join(out_dir, "qrels.tsv"), "w", encoding="utf-8") as fr:
        for r in ds.qrels_iter():
            rel = int(getattr(r, "relevance", 1) > 0)
            fr.write(f"{r.query_id}\t{r.doc_id}\t{rel}\n")
    print("[MrTyDi] wrote corpus.jsonl, queries.jsonl, qrels.tsv")

def load_jsonl_map(path: str, key="_id", text="text"):
    m={}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            j = json.loads(line); i = str(j.get(key) or j.get("id")); t=(j.get(text) or "").strip()
            if i and t: m[i]=t
    return m

def load_qrels_tsv(path: str):
    qrels=defaultdict(dict)
    with open(path, "r", encoding="utf-8") as f:
        for row in f:
            parts=row.strip().split("\t")
            if len(parts)<3: continue
            qid,did,rel=parts[0],parts[1],parts[2]
            try: r = int(float(rel))
            except: continue
            if r>0: qrels[qid][did]=1
    return dict(qrels)

def run_mrtydi():
    print("\n========== Mr.TyDi-RU ==========")
    prepare_mrtydi_ru(MRTYDI_DIR)
    # load
    corpus_raw = {}
    with open(os.path.join(MRTYDI_DIR,"corpus.jsonl"), "r", encoding="utf-8") as f:
        for line in f:
            j=json.loads(line); cid=str(j.get("_id") or j.get("id"))
            title=(j.get("title") or "").strip(); text=(j.get("text") or "").strip()
            corpus_raw[cid]=(title+" "+text).strip()
    queries = load_jsonl_map(os.path.join(MRTYDI_DIR,"queries.jsonl"))
    qrels   = load_qrels_tsv(os.path.join(MRTYDI_DIR,"qrels.tsv"))
    # subset: keep positives + fill negatives
    rng=random.Random(SEED)
    qids_all=[qid for qid in qrels if qid in queries]
    all_pos=set()
    for qid in qids_all: all_pos.update(qrels[qid].keys())
    pos_doc_ids=[d for d in all_pos if d in corpus_raw]
    if MRTYDI_DOC_CAP is None:
        doc_ids=list(corpus_raw.keys()); neg_added="N/A"
    else:
        if len(pos_doc_ids)>=MRTYDI_DOC_CAP:
            rng.shuffle(pos_doc_ids); doc_ids=pos_doc_ids[:MRTYDI_DOC_CAP]; neg_added=0
        else:
            need=MRTYDI_DOC_CAP-len(pos_doc_ids)
            neg_pool=[d for d in corpus_raw if d not in all_pos]; rng.shuffle(neg_pool)
            neg_added=min(need,len(neg_pool)); doc_ids=pos_doc_ids+neg_pool[:neg_added]
    corpus={d:corpus_raw[d] for d in doc_ids}
    doc_texts=[add_prefix(corpus[d],"passage") for d in doc_ids]
    qids=[qid for qid in qids_all if any((did in corpus) for did in qrels[qid])]
    if MRTYDI_FAST_Q and len(qids)>MRTYDI_FAST_Q:
        rng.shuffle(qids); qids=qids[:MRTYDI_FAST_Q]
    print(f"Docs={len(doc_ids):,} | Queries={len(qids):,} | PosDocs={len(pos_doc_ids):,} | NegAdded={neg_added}")

    loaders=[]
    tok_b, mdl_b, dev_b = safe_load_baseline(BASE_MODEL, DEVICE)
    loaders.append(("BASE", tok_b, mdl_b, dev_b, f"mrtydi__base__{corpus_signature(doc_ids)}__L{MAX_LEN}"))
    if FINETUNED_INST_DIR and os.path.isdir(FINETUNED_INST_DIR):
        tok_i, mdl_i, dev_i = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_INST_DIR, DEVICE)
        loaders.append(("FT_INST", tok_i, mdl_i, dev_i, f"mrtydi__inst__{corpus_signature(doc_ids)}__L{MAX_LEN}"))
    if FINETUNED_QUERY_DIR and os.path.isdir(FINETUNED_QUERY_DIR):
        tok_q, mdl_q, dev_q = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_QUERY_DIR, DEVICE)
        loaders.append(("FT_QUERY", tok_q, mdl_q, dev_q, f"mrtydi__qonly__{corpus_signature(doc_ids)}__L{MAX_LEN}"))

    # base query maps
    qmap = {qid: queries[qid] for qid in qids}
    results = {}

    for tag, tok, mdl, dev, sigbase in loaders:
        # plain
        q_plain = [add_prefix(qmap[q],"query") for q in qids]
        stats_plain, per_plain, _ = evaluate_model(f"MrTyDi | {tag} | plain", tok, mdl, dev, doc_ids, doc_texts, qids, q_plain, qrels, sigbase)

        # BANK (for p-MRR)
        q_bank  = [add_prefix((qmap[q]+" "+bank_for_qid(q)).strip(),"query") for q in qids]
        stats_bank, per_bank, _ = evaluate_model(f"MrTyDi | {tag} | q+BANK", tok, mdl, dev, doc_ids, doc_texts, qids, q_bank, qrels, sigbase)

        print(f"\n--- MrTyDi p-MRR (BANK − plain) :: {tag} ---")
        print({f"pMRR@{k}": round(paired_diff_mean(per_plain[k], per_bank[k]), 6) for k in K_LIST})

        # Prompt ablation
        if DO_PROMPT_ABLATION:
            print(f"\n--- Prompt ablation :: {tag} ---")
            for pname, prompt in PROMPTS_RU.items():
                qv = build_prompted_queries(qids, qmap, prompt)
                s,_,_ = evaluate_model(f"MrTyDi | {tag} | prompt:{pname}", tok, mdl, dev, doc_ids, doc_texts, qids, qv, qrels, sigbase)

        # Paraphrase robustness (mean/std)
        if DO_PARAPHRASE_ROBUST:
            print(f"\n--- Paraphrase robustness (5x) :: {tag} ---")
            suite = paraphrase_suite(qids, qmap, PARAPHRASES_RU)
            stats_list=[]
            for key, qv in suite.items():
                s,_,_ = evaluate_model(f"MrTyDi | {tag} | {key}", tok, mdl, dev, doc_ids, doc_texts, qids, qv, qrels, sigbase)
                stats_list.append(s)
            agg = agg_mean_std(stats_list)
            print("Paraphrase mean/std:", fmt(agg))

        # Noise stress
        if DO_NOISE_STRESS:
            print(f"\n--- Noise stress :: {tag} ---")
            suite = noise_suite(qids, qmap)
            base_stats=None
            for key, qv in suite.items():
                s,_,_ = evaluate_model(f"MrTyDi | {tag} | noise:{key}", tok, mdl, dev, doc_ids, doc_texts, qids, qv, qrels, sigbase)
                if key=="clean": base_stats=s
                else:
                    # report MRR@k drop vs clean
                    drops = {f"ΔMRR@{k}": round(s.get(f"MRR@{k}",0)-base_stats.get(f"MRR@{k}",0), 6) for k in K_LIST}
                    print(f"  drop vs clean ({key}):", drops)

        results[tag]=stats_plain  # keep at least plain for summary

        del mdl; torch.cuda.empty_cache()

    print("\n=== Mr.TyDi SUMMARY (plain only) ===")
    for k,v in results.items():
        print(k, "->", fmt(v))
    return results

# B) mFollowIR-RU

def run_mfollowir():
    print("\n========== mFollowIR-RU ==========")
    from datasets import load_dataset
    ds_q = load_dataset("jhu-clsp/mFollowIR-parquet","queries-rus")["queries"]
    ds_d = load_dataset("jhu-clsp/mFollowIR-parquet","corpus-rus")["corpus"]
    ds_og= load_dataset("jhu-clsp/mFollowIR-parquet","qrels_og-rus")["test"]

    doc_ids=[r["_id"] for r in ds_d]
    doc_txt=[add_prefix(((r.get("title") or "")+" "+(r.get("text") or "")).strip(),"passage") for r in ds_d]
    if MFOLLOWIR_DOC_CAP:
        rng=random.Random(SEED); idx=list(range(len(doc_ids))); rng.shuffle(idx); keep=sorted(idx[:MFOLLOWIR_DOC_CAP])
        doc_ids=[doc_ids[i] for i in keep]; doc_txt=[doc_txt[i] for i in keep]
    doc_set=set(doc_ids)
    qrels=defaultdict(dict)
    for r in ds_og:
        if r["corpus-id"] in doc_set and float(r["score"])>0:
            qrels[r["query-id"]][r["corpus-id"]]=1
    qrels={k:v for k,v in qrels.items() if v}

    qmap = {r["_id"]: r["text"].strip() for r in ds_q}
    qids = [qid for qid in qmap if qid in qrels]
    if MFOLLOWIR_FAST_Q and len(qids)>MFOLLOWIR_FAST_Q:
        rng=random.Random(SEED); rng.shuffle(qids); qids=qids[:MFOLLOWIR_FAST_Q]

    loaders=[]
    tok_b, mdl_b, dev_b = safe_load_baseline(BASE_MODEL, DEVICE)
    loaders.append(("BASE", tok_b, mdl_b, dev_b, f"mfollowir__base__{corpus_signature(doc_ids)}__L{MAX_LEN}"))
    if FINETUNED_INST_DIR and os.path.isdir(FINETUNED_INST_DIR):
        tok_i, mdl_i, dev_i = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_INST_DIR, DEVICE)
        loaders.append(("FT_INST", tok_i, mdl_i, dev_i, f"mfollowir__inst__{corpus_signature(doc_ids)}__L{MAX_LEN}"))
    if FINETUNED_QUERY_DIR and os.path.isdir(FINETUNED_QUERY_DIR):
        tok_q, mdl_q, dev_q = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_QUERY_DIR, DEVICE)
        loaders.append(("FT_QUERY", tok_q, mdl_q, dev_q, f"mfollowir__qonly__{corpus_signature(doc_ids)}__L{MAX_LEN}"))

    for tag, tok, mdl, dev, sigbase in loaders:
        # plain
        q_plain=[add_prefix(qmap[q],"query") for q in qids]
        stats_plain, per_plain, _ = evaluate_model(f"mFollowIR | {tag} | plain", tok, mdl, dev, doc_ids, doc_txt, qids, q_plain, qrels, sigbase)
        # BANK
        q_bank=[add_prefix((qmap[q]+" "+bank_for_qid(q)).strip(),"query") for q in qids]
        stats_bank, per_bank, _ = evaluate_model(f"mFollowIR | {tag} | q+BANK", tok, mdl, dev, doc_ids, doc_txt, qids, q_bank, qrels, sigbase)

        print(f"\n--- mFollowIR p-MRR (BANK − plain) :: {tag} ---")
        print({f"pMRR@{k}": round(paired_diff_mean(per_plain[k], per_bank[k]),6) for k in K_LIST})

        if DO_PROMPT_ABLATION:
            print(f"\n--- Prompt ablation :: {tag} ---")
            for pname, prompt in PROMPTS_RU.items():
                qv = build_prompted_queries(qids, qmap, prompt)
                evaluate_model(f"mFollowIR | {tag} | prompt:{pname}", tok, mdl, dev, doc_ids, doc_txt, qids, qv, qrels, sigbase)
        if DO_PARAPHRASE_ROBUST:
            print(f"\n--- Paraphrase robustness (5x) :: {tag} ---")
            suite=paraphrase_suite(qids, qmap, PARAPHRASES_RU)
            stats_list=[]
            for key,qv in suite.items():
                s,_,_ = evaluate_model(f"mFollowIR | {tag} | {key}", tok, mdl, dev, doc_ids, doc_txt, qids, qv, qrels, sigbase)
                stats_list.append(s)
            agg=agg_mean_std(stats_list); print("Paraphrase mean/std:", fmt(agg))
        if DO_NOISE_STRESS:
            print(f"\n--- Noise stress :: {tag} ---")
            suite=noise_suite(qids, qmap)
            base_stats=None
            for key,qv in suite.items():
                s,_,_ = evaluate_model(f"mFollowIR | {tag} | noise:{key}", tok, mdl, dev, doc_ids, doc_txt, qids, qv, qrels, sigbase)
                if key=="clean": base_stats=s
                else:
                    drops={f"ΔMRR@{k}": round(s.get(f"MRR@{k}",0)-base_stats.get(f"MRR@{k}",0),6) for k in K_LIST}
                    print(f"  drop vs clean ({key}):", drops)

        del mdl; torch.cuda.empty_cache()

    return {"done": True}

# C) LAReQA XQuAD-R RU↔EN

import requests
def ensure_lang_file(lang: str, out_dir: str) -> str:
    fp=os.path.join(out_dir, f"{lang}.json")
    if os.path.exists(fp): return fp
    url=f"https://raw.githubusercontent.com/google-research-datasets/lareqa/master/xquad-r/{lang}.json"
    print(f"[fetch] {lang}.json …"); r=requests.get(url,timeout=30); r.raise_for_status()
    with open(fp,"wb") as f: f.write(r.content); return fp

def split_sentences(context: str, sent_field):
    if isinstance(sent_field, list) and sent_field and isinstance(sent_field[0], list):
        sents=[]; 
        for s,e in sent_field:
            try:
                t=context[s:e].strip()
                if t: sents.append(t)
            except: pass
        return sents
    if isinstance(sent_field, list) and sent_field and isinstance(sent_field[0], str):
        return [s.strip() for s in sent_field if str(s).strip()]
    return re.split(r'\s*(?<=\.|\?|!)\s+', context.strip())

def load_xquadr(path: str):
    with open(path,"r",encoding="utf-8") as f: data=json.load(f)
    qa_map={}; sents=[]
    for art in data.get("data",[]):
        for par in art.get("paragraphs",[]):
            context=par.get("context","") or ""
            ss=split_sentences(context, par.get("sentences")); base=len(sents); sents.extend(ss)
            for qa in par.get("qas",[]):
                qid=qa.get("id") or qa.get("qid") or ""
                qtext=qa.get("question","").strip()
                answers=[a.get("text","").strip() for a in qa.get("answers",[]) if str(a.get("text","")).strip()]
                qa_map[qid]={"question":qtext,"answers":answers}
    return sents, qa_map

def normalize(s): return re.sub(r"\s+"," ", s.strip().lower())

def build_xling_ir(qas_src, answers_tgt, cand_sents, cand_lang: str, doc_cap: int, fast_q: int):
    doc_ids=[f"{cand_lang}-s{ix}" for ix in range(len(cand_sents))]
    if doc_cap and doc_cap<len(doc_ids):
        rng=random.Random(SEED); idx=list(range(len(doc_ids))); rng.shuffle(idx); keep=sorted(idx[:doc_cap])
        cand_sents=[cand_sents[i] for i in keep]; doc_ids=[doc_ids[i] for i in keep]
    qids_all=sorted(set(qas_src.keys()) & set(answers_tgt.keys()))
    if fast_q and fast_q<len(qids_all):
        rng=random.Random(SEED); rng.shuffle(qids_all); qids_all=sorted(qids_all[:fast_q])
    q_texts={qid: qas_src[qid]["question"] for qid in qids_all}
    cand_norm=[normalize(x) for x in cand_sents]
    qrels=defaultdict(dict)
    for qid in qids_all:
        golds=[normalize(a) for a in answers_tgt[qid].get("answers",[]) if a.strip()]
        if not golds: continue
        for i,s in enumerate(cand_norm):
            if any(g and g in s for g in golds):
                qrels[qid][doc_ids[i]]=1
    qids=[qid for qid in qids_all if qrels.get(qid)]
    qrels={qid:qrels[qid] for qid in qids}
    return doc_ids, cand_sents, qids, q_texts, qrels

def run_lareqa():
    print("\n========== LAReQA XQuAD-R ==========")
    ru_path=ensure_lang_file("ru", LAREQA_DIR); en_path=ensure_lang_file("en", LAREQA_DIR)
    ru_sents, ru_qas = load_xquadr(ru_path); en_sents, en_qas = load_xquadr(en_path)

    def run_dir(tag, qas_src, ans_tgt, cand_sents, cand_lang, prompts, paraphrases):
        doc_ids, docs, qids, qmap, qrels = build_xling_ir(qas_src, ans_tgt, cand_sents, cand_lang, LAREQA_DOC_CAP, LAREQA_FAST_Q)
        print(f"\n--- {tag} --- Docs={len(doc_ids):,} | Queries={len(qids):,} | QrelsQ={len(qrels):,}")
        doc_texts=[add_prefix(t,"passage") for t in docs]
        loaders=[]
        tok_b, mdl_b, dev_b = safe_load_baseline(BASE_MODEL, DEVICE)
        loaders.append(("BASE", tok_b, mdl_b, dev_b, f"lareqa_{tag}__base__{corpus_signature(doc_ids)}__L{MAX_LEN}"))
        if FINETUNED_INST_DIR and os.path.isdir(FINETUNED_INST_DIR):
            tok_i, mdl_i, dev_i = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_INST_DIR, DEVICE)
            loaders.append(("FT_INST", tok_i, mdl_i, dev_i, f"lareqa_{tag}__inst__{corpus_signature(doc_ids)}__L{MAX_LEN}"))
        if FINETUNED_QUERY_DIR and os.path.isdir(FINETUNED_QUERY_DIR):
            tok_q, mdl_q, dev_q = safe_load_peft_for_inference(BASE_MODEL, FINETUNED_QUERY_DIR, DEVICE)
            loaders.append(("FT_QUERY", tok_q, mdl_q, dev_q, f"lareqa_{tag}__qonly__{corpus_signature(doc_ids)}__L{MAX_LEN}"))

        for mtag, tok, mdl, dev, sigbase in loaders:
            # plain
            q_plain=[add_prefix(qmap[q],"query") for q in qids]
            stats_plain, per_plain, _ = evaluate_model(f"LAReQA {tag} | {mtag} | plain", tok, mdl, dev, doc_ids, doc_texts, qids, q_plain, qrels, sigbase)
            # BANK
            q_bank=[add_prefix((qmap[q]+" "+bank_for_qid(q)).strip(),"query") for q in qids]
            stats_bank, per_bank, _ = evaluate_model(f"LAReQA {tag} | {mtag} | q+BANK", tok, mdl, dev, doc_ids, doc_texts, qids, q_bank, qrels, sigbase)
            print(f"\n--- LAReQA p-MRR (BANK − plain) :: {tag} :: {mtag} ---")
            print({f"pMRR@{k}": round(paired_diff_mean(per_plain[k], per_bank[k]),6) for k in K_LIST})

            if DO_PROMPT_ABLATION:
                print(f"\n--- Prompt ablation :: {tag} :: {mtag} ---")
                for pname, prompt in prompts.items():
                    qv = build_prompted_queries(qids, qmap, prompt)
                    evaluate_model(f"LAReQA {tag} | {mtag} | prompt:{pname}", tok, mdl, dev, doc_ids, doc_texts, qids, qv, qrels, sigbase)
            if DO_PARAPHRASE_ROBUST:
                print(f"\n--- Paraphrase robustness (5x) :: {tag} :: {mtag} ---")
                suite=paraphrase_suite(qids, qmap, paraphrases)
                stats_list=[]
                for key,qv in suite.items():
                    s,_,_ = evaluate_model(f"LAReQA {tag} | {mtag} | {key}", tok, mdl, dev, doc_ids, doc_texts, qids, qv, qrels, sigbase)
                    stats_list.append(s)
                agg=agg_mean_std(stats_list); print("Paraphrase mean/std:", fmt(agg))
            if DO_NOISE_STRESS:
                print(f"\n--- Noise stress :: {tag} :: {mtag} ---")
                suite=noise_suite(qids, qmap)
                base_stats=None
                for key,qv in suite.items():
                    s,_,_ = evaluate_model(f"LAReQA {tag} | {mtag} | noise:{key}", tok, mdl, dev, doc_ids, doc_texts, qids, qv, qrels, sigbase)
                    if key=="clean": base_stats=s
                    else:
                        drops={f"ΔMRR@{k}": round(s.get(f"MRR@{k}",0)-base_stats.get(f"MRR@{k}",0),6) for k in K_LIST}
                        print(f"  drop vs clean ({key}):", drops)
            del mdl; torch.cuda.empty_cache()

    # RU→EN: RU queries -> use RU prompts/paraphrases
    run_dir("RU→EN", ru_qas, en_qas, en_sents, "en", PROMPTS_RU, PARAPHRASES_RU)
    # EN→RU: EN queries -> use EN prompts/paraphrases
    run_dir("EN→RU", en_qas, ru_qas, ru_sents, "ru", PROMPTS_EN, PARAPHRASES_EN)
    return {"done": True}


all_results = {}
all_results["MrTyDi-RU"]  = run_mrtydi()
all_results["mFollowIR"]  = run_mfollowir()
all_results["LAReQA"]     = run_lareqa()

print("\n=============== DONE ===============")


Device: cuda | K=[1, 5, 10] | MAX_LEN=256

========== Mr.TyDi-RU ==========
[MrTyDi] prepared files already present.
Docs=20,000 | Queries=995 | PosDocs=1,100 | NegAdded=18900

=== MrTyDi | BASE | plain ===


[cache] docs torch.Size([20000, 768]) saved -> /kaggle/working/robust_eval_cache/docs__mrtydi__base__363db7cc__L256.pt


{'N': 995, 'Hit@1': 0.808, 'Recall@1': 0.808, 'MRR@1': 0.808, 'MAP@1': 0.808, 'nDCG@1': 0.808, 'Recall@5': 0.9357, 'MRR@5': 0.862, 'MAP@5': 0.8446, 'nDCG@5': 0.8686, 'Recall@10': 0.9588, 'MRR@10': 0.8653, 'MAP@10': 0.8494, 'nDCG@10': 0.8782}

=== MrTyDi | BASE | q+BANK ===
[cache] docs torch.Size([20000, 768]) loaded


{'N': 995, 'Hit@1': 0.7859, 'Recall@1': 0.7859, 'MRR@1': 0.7859, 'MAP@1': 0.7859, 'nDCG@1': 0.7859, 'Recall@5': 0.9095, 'MRR@5': 0.8357, 'MAP@5': 0.8151, 'nDCG@5': 0.8394, 'Recall@10': 0.9347, 'MRR@10': 0.8391, 'MAP@10': 0.8203, 'nDCG@10': 0.85}

--- MrTyDi p-MRR (BANK − plain) :: BASE ---
{'pMRR@1': -0.022111, 'pMRR@5': -0.026382, 'pMRR@10': -0.026228}

--- Prompt ablation :: BASE ---

=== MrTyDi | BASE | prompt:plain ===
[cache] docs torch.Size([20000, 768]) loaded


{'N': 995, 'Hit@1': 0.808, 'Recall@1': 0.808, 'MRR@1': 0.808, 'MAP@1': 0.808, 'nDCG@1': 0.808, 'Recall@5': 0.9357, 'MRR@5': 0.862, 'MAP@5': 0.8446, 'nDCG@5': 0.8686, 'Recall@10': 0.9588, 'MRR@10': 0.8653, 'MAP@10': 0.8494, 'nDCG@10': 0.8782}

=== MrTyDi | BASE | prompt:minimal_ru ===
[cache] docs torch.Size([20000, 768]) loaded


...

{'N': 1188, 'Hit@1': 0.7256, 'Recall@1': 0.7256, 'MRR@1': 0.7256, 'MAP@1': 0.7256, 'nDCG@1': 0.7256, 'Recall@5': 0.9242, 'MRR@5': 0.805, 'MAP@5': 0.7011, 'nDCG@5': 0.7509, 'Recall@10': 0.9554, 'MRR@10': 0.8093, 'MAP@10': 0.7035, 'nDCG@10': 0.7604}
Paraphrase mean/std: {'Hit@1_mean': 0.7034, 'Hit@1_std': 0.0162, 'Recall@1_mean': 0.7034, 'Recall@1_std': 0.0162, 'MRR@1_mean': 0.7034, 'MRR@1_std': 0.0162, 'MAP@1_mean': 0.7034, 'MAP@1_std': 0.0162, 'nDCG@1_mean': 0.7034, 'nDCG@1_std': 0.0162, 'Recall@5_mean': 0.9136, 'Recall@5_std': 0.0073, 'MRR@5_mean': 0.7868, 'MRR@5_std': 0.0125, 'MAP@5_mean': 0.6835, 'MAP@5_std': 0.0122, 'nDCG@5_mean': 0.7344, 'nDCG@5_std': 0.0113, 'Recall@10_mean': 0.9508, 'Recall@10_std': 0.0025, 'MRR@10_mean': 0.792, 'MRR@10_std': 0.0119, 'MAP@10_mean': 0.6866, 'MAP@10_std': 0.0117, 'nDCG@10_mean': 0.7456, 'nDCG@10_std': 0.0099}

--- Noise stress :: EN→RU :: FT_QUERY ---

=== LAReQA EN→RU | FT_QUERY | noise:clean ===
[cache] docs torch.Size([1219, 768]) loaded


{'N': 1188, 'Hit@1': 0.7222, 'Recall@1': 0.7222, 'MRR@1': 0.7222, 'MAP@1': 0.7222, 'nDCG@1': 0.7222, 'Recall@5': 0.9285, 'MRR@5': 0.8044, 'MAP@5': 0.7005, 'nDCG@5': 0.7511, 'Recall@10': 0.9588, 'MRR@10': 0.8086, 'MAP@10': 0.7027, 'nDCG@10': 0.7602}

=== LAReQA EN→RU | FT_QUERY | noise:nopunct ===
[cache] docs torch.Size([1219, 768]) loaded


{'N': 1188, 'Hit@1': 0.7298, 'Recall@1': 0.7298, 'MRR@1': 0.7298, 'MAP@1': 0.7298, 'nDCG@1': 0.7298, 'Recall@5': 0.9209, 'MRR@5': 0.8067, 'MAP@5': 0.7013, 'nDCG@5': 0.7504, 'Recall@10': 0.9579, 'MRR@10': 0.8119, 'MAP@10': 0.7042, 'nDCG@10': 0.7614}
  drop vs clean (nopunct): {'ΔMRR@1': 0.007576, 'ΔMRR@5': 0.002287, 'ΔMRR@10': 0.003349}

=== LAReQA EN→RU | FT_QUERY | noise:swap ===
[cache] docs torch.Size([1219, 768]) loaded


{'N': 1188, 'Hit@1': 0.7189, 'Recall@1': 0.7189, 'MRR@1': 0.7189, 'MAP@1': 0.7189, 'nDCG@1': 0.7189, 'Recall@5': 0.9158, 'MRR@5': 0.7966, 'MAP@5': 0.6922, 'nDCG@5': 0.742, 'Recall@10': 0.9537, 'MRR@10': 0.8018, 'MAP@10': 0.6956, 'nDCG@10': 0.7537}
  drop vs clean (swap): {'ΔMRR@1': -0.003367, 'ΔMRR@5': -0.007814, 'ΔMRR@10': -0.006763}

=== LAReQA EN→RU | FT_QUERY | noise:lower ===
[cache] docs torch.Size([1219, 768]) loaded


{'N': 1188, 'Hit@1': 0.7298, 'Recall@1': 0.7298, 'MRR@1': 0.7298, 'MAP@1': 0.7298, 'nDCG@1': 0.7298, 'Recall@5': 0.92, 'MRR@5': 0.8045, 'MAP@5': 0.7012, 'nDCG@5': 0.7496, 'Recall@10': 0.9537, 'MRR@10': 0.8094, 'MAP@10': 0.7048, 'nDCG@10': 0.7611}
  drop vs clean (lower): {'ΔMRR@1': 0.007576, 'ΔMRR@5': 9.8e-05, 'ΔMRR@10': 0.000762}

=============== DONE ===============
